In [1]:
# %matplotlib widget
# %matplotlib inline
%matplotlib qt5

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

from qick import *
from qick.helpers import gauss

import time
import os
import sys
sys.path.append('/home/xilinx/jupyter_notebooks/')
import scipy as sp
import json
from scipy.fft import fft, fftfreq

from slab.instruments import *
from slab.experiment import Experiment
from slab.datamanagement import SlabFile
from slab import get_next_filename, AttrDict

# Figure params
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [10,6]
plt.rcParams.update({'font.size': 14})
mpl.rcParams['figure.dpi'] = 150

import Pyro4.util

default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
linestyle_cycle=['solid', 'dashed', 'dotted', 'dashdot']
marker_cycle = ['o', '*', 's', '^']

Could not import QickInstrument (will only work if running on rfsoc). If running on rfsoc, try install/update qick package
Warning failed to import AWG81180A
Warning failed to import Tek5014
Warning failed to import Tek70001
Warning failed to import M8195A


# Load experiments

In [3]:
# %aimport experiments
# NOTE: ADDING NEW METHODS WILL NOT BE UPDATED
import experiments as meas

imported experiments.clifford_averager_program
imported experiments.fitting
imported experiments.four_qubit.fourQ_state_tomo
imported experiments.qram_protocol_timestepped
imported experiments.single_qubit.acstarkshift_spectroscopy
imported experiments.single_qubit.amplitude_rabi
imported experiments.single_qubit.length_rabi
imported experiments.single_qubit.pulse_probe_spectroscopy
imported experiments.single_qubit.resonator_spectroscopy
imported experiments.single_qubit.rfsoc_tof_calibration
imported experiments.single_qubit.single_shot
imported experiments.single_qubit.t1
imported experiments.single_qubit.t2_ramsey
imported experiments.three_qubit.threeQ_state_tomo
imported experiments.two_qubit.amplitude_rabi_EgGf
imported experiments.two_qubit.amplitude_rabi_f0g1
imported experiments.two_qubit.amprabi_opt_ctrl_state_prep
imported experiments.two_qubit.crosstalk_echo_calib
imported experiments.two_qubit.length_rabi_EgGf
imported experiments.two_qubit.length_rabi_F0G1
imported exper

### Set save data

In [4]:
save_data = False

In [5]:
def prev_data(expt_path, filename=None):
    """Reopen saved data"""
    temp_data_file = expt_path
    if filename is not None: temp_data_file = os.path.join(expt_path, filename)
    print(temp_data_file)
    with SlabFile(temp_data_file) as a:
        attrs = dict()
        for key in list(a.attrs):
            attrs.update({key:json.loads(a.attrs[key])})
        keys = list(a)
        temp_data = dict()
        for key in keys:
            temp_data.update({key:np.array(a[key])})
        print(f'Extracted data and attrs {list(a.attrs)}')
    return temp_data, attrs

In [6]:
# expt_path=os.path.join(os.getcwd(), 'data', 'data_240617')
data_path = 'S:\\QRAM\\qram_4QR2\\'
expt_path = os.path.join(data_path, 'data', 'data_241025')
print('Data will be stored in', expt_path)

Data will be stored in S:\QRAM\qram_4QR2\data\data_241025


In [7]:
# config_file = 'config_zcu216.yml'
# config_file = 'config_q3diamond.yml'
# config_file = 'config_q3diamond_full688_reset.yml'
config_file = 'config_q3diamond_full688and638_reset.yml'
config_path = os.path.join('s:\\Connie\\experiments\\qram_tprocv1_expts\\configs\\', config_file)
print('Config will be', config_path)

Config will be s:\Connie\experiments\qram_tprocv1_expts\configs\config_q3diamond_full688and638_reset.yml


# Load config

In [145]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

# Setup experiment hardware

Set in config:
- LO alias, frequency, power
- SOC dac channels, nyquist zones

In [9]:
# im = InstrumentManager(ns_address='192.168.14.1') # Chicago lab
# im = InstrumentManager(ns_address='192.168.137.1') # SLAC lab
# im = InstrumentManager(ns_address='10.108.30.56') # Spilker Fermium
im = InstrumentManager(ns_address='10.108.30.75') # Spilker Vibranium

print(list(im))
# print(im)

['Pyro.NameServer', 'Qick_QRAM']


In [10]:
soc = QickConfig(im[yaml_cfg['aliases']['soc']].get_cfg())
print(soc)
# print(soc.description())
# print(soc.dump_cfg())

# soc = QickConfig(im['Qick187'].get_cfg())

QICK running on ZCU216, software version 0.2.272

Firmware configuration (built Sat Oct  5 10:21:47 2024):

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	7 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 1 is 1_230, on JHC4
	2:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 2 is 2_230, on JHC3
	3:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 3 is 3_230, on JHC4
	4:	axis_signal_gen_v6 - envelope memory 32768 samples (4.762 us)
		fs=6881.280 MHz, fabric=430.080 MH

In [11]:
lo1 = im[yaml_cfg.aliases.readout_LO]
lo1.open_device()
print(lo1.get_id())

# TURN OFF
lo1.set_standby(True)
lo1.set_output_state(False)

# # TURN ON
# lo1.set_standby(False)
# lo1.set_output_state(True)
# lo_freq = float(yaml_cfg.hw.lo.readout.frequency)
# lo_power = float(yaml_cfg.hw.lo.readout.power)
# lo1.set_frequency(lo_freq) # Set LO frequency
# print(f'set freq to [MHz] {lo_freq*1e-6}')
# lo1.set_power(lo_power) # Set LO power
# print('set power to', lo_power)

KeyError: 'SC_20A1'

## Set TWPA parameters

Set in config:
- LO alias, frequency, power
- SOC dac channels, nyquist zones

In [ ]:
"""
TWPA ON
"""

print('Using config file', config_file)
with open(config_file, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

im = InstrumentManager(ns_address='10.108.30.56') # Spilker Fermium

lo_twpa = im[yaml_cfg.aliases.twpa_LO]
# lo_twpa = SignalCore(name="SignalCore", address='10003494')
lo_twpa.open_device()
print(lo_twpa.get_id())

# TURN ON
lo_twpa.set_standby(False)
lo_twpa.set_output_state(True)
twpa_freq = float(yaml_cfg.hw.lo.twpa.frequency)
twpa_power = float(yaml_cfg.hw.lo.twpa.power)
lo_twpa.set_frequency(twpa_freq) # Set LO frequency
print(f'set twpa pump freq to {twpa_freq*1e-6} MHz')
lo_twpa.set_power(twpa_power) # Set LO power
print(f'set twpa pump power to {twpa_power} dBm')

In [ ]:
"""
TWPA OFF
"""
im = InstrumentManager(ns_address='10.108.30.56') # Spilker Fermium

lo_twpa = im[yaml_cfg.aliases.twpa_LO]
# lo_twpa = SignalCore(name="SignalCore", address='10003494')
lo_twpa.open_device()
print(lo_twpa.get_id())

# TURN OFF
lo_twpa.set_standby(True)
lo_twpa.set_output_state(False)

# Qubit/resonator to probe

In [35]:
all_results = [dict(), dict(), dict(), dict()]
saved_files = []

In [152]:
qubit_i = 0
results_Qi = all_results[qubit_i]

In [37]:
resonator_reset = [0, 2, 3]
full_mux_expt = True
use_robust_pulses = True

## Set reps and rounds

In [12]:
reps = 100
rounds = 10

# RFSoC Calibration

In [ ]:
tof = meas.ToFCalibrationExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"adc_trig_offset_calibration_qubit{qubit_i}",
    config_file=config_path,
)

tof.cfg.expt = dict(
    pulse_length=0.5, # [us]
    readout_length=1.0, # [us]
    trig_offset=0, # [clock ticks]
    # gain=32000,
    gain=0.8, # [MUX units: -1 to 1]
    frequency=tof.cfg.device.readout.frequency[qubit_i]-15, # [MHz]
    reps=reps*rounds, # Number of averages per point
    # qubit=qubit_i,
    qubit=qubit_i,
)
tof.cfg.device.readout.relax_delay=0.1 # wait time between experiments [us]
# print(tof.cfg)

try:
    tof.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
tof.display(adc_trig_offset=150)

# Initial single tone resonator spectroscopy

In [49]:
rspec = meas.ResonatorSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"resonator_spectroscopy_qubit{qubit_i}",
    config_file=config_path,
)
rspec.cfg = AttrDict(deepcopy(yaml_cfg))

span = 1.0 # MHz
npts = 150
rspec.cfg.expt = dict(
    start=rspec.cfg.device.readout.frequency[qubit_i]-span/2, # [MHz]
    step=span/npts,
    expts=npts,
    reps=reps*rounds,
    pulse_e=False, # add ge pi pulse prior to measurement
    pulse_f=False, # add ef pi pulse prior to measurement
    qTest=qubit_i,
    qZZ=None,

    resonator_reset=resonator_reset,
    full_mux_expt=full_mux_expt,
)
# rspec.cfg.device.readout.gain[qubit_i] /= 3
# rspec.cfg.device.readout.readout_length = [3*rspec.cfg.device.readout.readout_length[qubit_i]]*4

rspec.cfg.device.readout.relax_delay = 25 # Wait time between experiments [us] - should be 5x resonator lifetime for good shape

try:
    rspec.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/150 [00:00<?, ?it/s]

In [50]:
# TESTING
# # rspec = meas.ResonatorSpectroscopyExperiment(config_file=config_path,) 
# temp_data, attrs = prev_data(expt_path, '00001_resonator_spectroscopy_qubit2.h5')
# rspec.data = temp_data

rspec.data = rspec.analyze(fit=True)
rspec_fit = rspec.data['fit']
# if not isinstance(rspec_fit, (list, np.ndarray)):
#     rspec.display(fit=False)
#     assert False, 'Fit failed, please manually fix!'
rspec.display(fit=True)
# yaml_cfg.device.readout.frequency[qubit_i] = rspec_fit[0]

res_freq_g = rspec.data['xpts'][np.argmin(rspec.data['amps'])]
# res_freq_g = rspec.data['xpts'][np.argmax(rspec.data['amps'])]
yaml_cfg.device.readout.frequency[qubit_i] = res_freq_g

print(f'*Set cfg res {qubit_i} freq to {yaml_cfg.device.readout.frequency[qubit_i]}*')
results_Qi.update({'Res freq':float(yaml_cfg.hw.lo.readout.frequency)*1e-6 + yaml_cfg.device.readout.lo_sideband[qubit_i]*(yaml_cfg.device.readout.frequency[qubit_i] + yaml_cfg.hw.soc.dacs.readout.mixer_freq[qubit_i])})
results_Qi.update({'Res freq (yaml)':yaml_cfg.device.readout.frequency[qubit_i]})
results_Qi.update({'Res kappa':abs(rspec_fit[0]*(1/rspec_fit[1] + 1/rspec_fit[2]))})
print(f'qubit {qubit_i}', results_Qi)

electrical delay phase correction (no mux support) (deg) 6.258447188384258
param guess [6971.096666666666, 34855.48333333333, 34855.48333333333, 0, -0.022715829751278505, 0.8418322311653272]
param bounds ([6970.606666666667, 0, 0, -6.283185307179586, -0.03407374462691776, -inf], [6971.586666666667, 1000000000.0, 100000000.0, 6.283185307179586, 0.03407374462691776, 1.0278030397182794])
popt [ 6.97108993e+03  1.21028854e+05  5.05096512e+04  4.42167625e-02
 -3.06900828e-02 -4.50771994e-03]

Freq with minimum transmission: -278.91333333333296
Freq with maximum transmission: -279.37999999999965
From fit:
	f0: 6971.089931704766
	Qi: 121028.85404003673 	 kappa_i/2pi: 0.05759857834726508
	Qe: 50509.651196300525 	 kappa_e/2pi: 0.1380150083518167
	Q0: 35637.04367032849
	kappa [MHz]: 0.1956135866990818
	phi [radians]: 0.04421676250641176
baseline 292.1581354807663
*Set cfg res 3 freq to -278.9066666666663*
qubit 3 {'Res freq': 6971.093333333334, 'Res freq (yaml)': -278.9066666666663, 'Res kappa':

In [135]:
if save_data: saved_files.append(rspec.save_data())

# Qubit ge spectroscopy

## Pulse Probe (qubit) spectroscopy

In [16]:
qspec = meas.PulseProbeSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"qubit_spectroscopy_qubit{qubit_i}",
    config_file=config_path,
)
qspec.cfg = AttrDict(deepcopy(yaml_cfg))

span = 50
npts = 250

qspec.cfg.expt = dict(
    start=np.reshape(yaml_cfg.device.qubit.f_ge, (4,4))[qubit_i, qubit_i]-span/2, # [MHz]
    step=span/npts,
    expts=npts,
    reps=reps, # Number of averages per point
    rounds=rounds, # Number of start to finish sweeps to average over
    length=1, # Qubit probe constant pulse length [us]
    gain=int(np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i]/10), # Qubit pulse gain
    pulse_type='gauss',
    qTest=qubit_i,
    qZZ=None,
    checkEF=False,
)
qspec.cfg.device.readout.relax_delay = 10 # Wait time between experiments [us]
qspec.go(analyze=False, display=False, progress=True, save=False)

Running pulse probe on Q3 


  0%|          | 0/10 [00:00<?, ?it/s]

In [17]:
# # TESTING
# # qspec = meas.PulseProbeSpectroscopyExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00001_qubit_spectroscopy_qubit0.h5')
# qspec.data = temp_data

best_signs = [1, 0, 0]
best_fit_err_i = np.inf
best_fit_err_q = np.inf
for sign in [1, -1]:
    try: 
        qspec.analyze(fit=True, signs=[1, sign, sign])
        fit_err_i = qspec.data['fit_err_avgi'][2][2]
        fit_err_q = qspec.data['fit_err_avgq'][2][2]
        if fit_err_i < best_fit_err_i:
            best_signs[1] = sign
            best_fit_err_i = fit_err_i
        if fit_err_q < best_fit_err_q:
            best_signs[2] = sign
            best_fit_err_q = fit_err_q
    except: continue
if best_fit_err_i == np.inf and best_fit_err_q == np.inf:
    qspec.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

qspec.analyze(fit=True, signs=best_signs)
qspec.display(fit=True, signs=best_signs)

qspec_fit, qspec_fit_err = meas.fitting.get_best_fit(qspec.data)
yaml_cfg.device.qubit.f_ge = np.reshape(yaml_cfg.device.qubit.f_ge, (4,4))
yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] = qspec_fit[2]
print(f'*Set qubit {qubit_i} f_ge to {yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i]}*')
results_Qi.update({'Qubit freq':yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
yaml_cfg.device.qubit.f_ge = yaml_cfg.device.qubit.f_ge.flatten().tolist()
print(results_Qi)

Found peak in amps at [MHz] 4380.96234875197, HWHM 2.023586589976097
Found peak in I at [MHz] 4380.916145567329, HWHM 2.2553416752914712
Found peak in Q at [MHz] 4381.035808362081, HWHM 1.8906706634981154
best 0 0.03869489590432924
*Set qubit 3 f_ge to 4380.96234875197*
{'Res freq': 6971.132559436666, 'Res freq (yaml)': -228.86744056333333, 'Res kappa': 0.20429779230466372, 'Qubit freq': 4380.96234875197}


In [138]:
if save_data: saved_files.append(qspec.save_data())

## Pulse Probe (qubit) spectroscopy round 2

In [18]:
qspec2 = meas.PulseProbeSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"qubit_spectroscopy_qubit{qubit_i}",
    config_file=config_path,
)
qspec2.cfg = AttrDict(deepcopy(yaml_cfg))

span = 15
npts = 250

qspec2.cfg.expt = dict(
    start=np.reshape(qspec2.cfg.device.qubit.f_ge, (4,4))[qubit_i, qubit_i]-span/2, # [MHz]
    step=span/npts,
    expts=npts,
    reps=reps, # Number of averages per point
    rounds=rounds, # Number of start to finish sweeps to average over
    length=1, # Qubit probe constant pulse length [us]
    gain=int(np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i]/80), # Qubit pulse gain
    pulse_type='gauss',
    qTest=qubit_i,
    qZZ=None,
    checkEF=False,
)
qspec2.cfg.device.readout.relax_delay = 10 # Wait time between experiments [us]
qspec2.go(analyze=False, display=False, progress=True, save=False)

Running pulse probe on Q3 


  0%|          | 0/10 [00:00<?, ?it/s]

In [19]:
# # TESTING
# # qspec2 = meas.PulseProbeSpectroscopyExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00001_qubit_spectroscopy_qubit0.h5')
# qspec2.data = temp_data

best_signs = [1, 0, 0]
best_fit_err_i = np.inf
best_fit_err_q = np.inf
for sign in [1, -1]:
    try: 
        qspec2.analyze(fit=True, signs=[1, sign, sign])
        fit_err_amps = qspec2.data['fit_err_amps'][2][2]
        fit_err_i = qspec2.data['fit_err_avgi'][2][2]
        fit_err_q = qspec2.data['fit_err_avgq'][2][2]
        if fit_err_i < best_fit_err_i:
            best_signs[1] = sign
            best_fit_err_i = fit_err_i
        if fit_err_q < best_fit_err_q:
            best_signs[2] = sign
            best_fit_err_q = fit_err_q
    except: continue
if fit_err_amps == np.inf and best_fit_err_i == np.inf and best_fit_err_q == np.inf:
    qspec2.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

qspec2.analyze(fit=True, signs=best_signs)
qspec2.display(fit=True, signs=best_signs)

qspec2_fit, qspec2_fit_err = meas.fitting.get_best_fit(qspec2.data)
yaml_cfg.device.qubit.f_ge = np.reshape(yaml_cfg.device.qubit.f_ge, (4,4))
yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] = qspec2_fit[2]
qubit_hwhm = qspec2_fit[3]
print(f'*Set qubit {qubit_i} f_ge to {yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i]}*')
results_Qi.update({'Qubit freq':yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
yaml_cfg.device.qubit.f_ge = yaml_cfg.device.qubit.f_ge.flatten().tolist()
print(results_Qi)

Found peak in amps at [MHz] 4380.914774648676, HWHM 1.3085694089383166
Found peak in I at [MHz] 4380.868677035046, HWHM 1.4452405703563218
Found peak in Q at [MHz] 4380.966700685872, HWHM 1.4103396452867274
best 0 0.03263221728838213
*Set qubit 3 f_ge to 4380.914774648676*
{'Res freq': 6971.132559436666, 'Res freq (yaml)': -228.86744056333333, 'Res kappa': 0.20429779230466372, 'Qubit freq': 4380.914774648676}


In [141]:
if save_data: saved_files.append(qspec2.save_data())

## Length Rabi

In [78]:
lengthrabi = meas.LengthRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"length_rabi_qubit{qubit_i}",
    config_file=config_path,
)
lengthrabi.cfg = AttrDict(deepcopy(yaml_cfg))

lengthrabi.cfg.expt = dict(
    start=0, # pulse length (sigma) [us]
    step=soc.cycles2us(1), # [us]
    # step=0.001,
    expts=121,
    reps=reps*rounds,
    pulse_type='gauss',
    checkEF=False, 
    qTest=qubit_i,
    qZZ=None,
)
# lengthrabi.cfg.device.readout.relax_delay = 50 # Wait time between experiments [us]
# lengthrabi.pi_ge.gain[4*qubit_i+qubit_i] = 5000 # qubit gain [DAC units],

try:
    lengthrabi.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

Running length rabi on Q2 


  0%|          | 0/121 [00:00<?, ?it/s]

In [79]:
# # TESTING
# # lengthrabi = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# lengthrabi.data = temp_data

lengthrabi.analyze(fit=True)

lengthrabi_fit, lengthrabi_fit_err = meas.fitting.get_best_fit(lengthrabi.data) #, fitfunc=fitter.decaysin)

lengthrabi.display(fit=True)
# p = lengthrabi_fit
# if p[2] > 180: p[2] = p[2] - 360
# elif p[2] < -180: p[2] = p[2] + 360
# if p[2] < 0: pi_len = (1/2 - p[2]/180)/2/p[1]
# else: pi_len = (3/2 - p[2]/180)/2/p[1]
# pi_len = np.max((0.9*pi_len, 0.015)) # since we run the length rabi at a < max gain

# lengthrabi_decay = p[3]

# print(f'Will use sigma {pi_len} for amplitude rabi')

best 0 0.3211168491826572
Pi length from avgi data [us]: 0.11257977810532821
	Pi/2 length from avgi data [us]: 0.056289889052664105

Pi length from avgq data [us]: 0.09888036150893775
Pi/2 length from avgq data [us]: 0.049440180754468877


In [ ]:
# if save_data: saved_files.append(lengthrabi.save_data())

## Amplitude Rabi

In [15]:
pi_len = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]
# pi_len = 0.008
print('qubit', qubit_i, 'pi_len', pi_len)

qubit 1 pi_len 0.025


In [24]:
amprabi = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi.cfg = AttrDict(deepcopy(yaml_cfg))

span = min((32000, int(1.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i])))
# span = 32000
npts = 151

amprabi.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    rounds=rounds,
    sigma_test=pi_len, # gaussian sigma for pulse length - overrides config [us]
    checkEF=False, 
    qTest=qubit_i,
    qZZ=None,
    pulse_type='gauss', 

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    # use_robust_pulses=use_robust_pulses,
)
amprabi.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi on Q1 


  0%|          | 0/10 [00:00<?, ?it/s]

(4, 151, 100)


In [25]:
# # TESTING
# # amprabi = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi.data = temp_data

# amprabi.display(fit=False)

amprabi.analyze(fit=True)

amprabi_fit, amprabi_fit_err = meas.fitting.get_best_fit(amprabi.data)

p = amprabi_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi.display(fit=False)
    print(f'WARNING: Fit pi pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi.display(fit=True)
yaml_cfg.device.qubit.pulses.pi_ge.sigma[qubit_i] = amprabi.cfg.expt.sigma_test

print(f'*Set pi pulse sigma for qubit {qubit_i} to {np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]}*')
yaml_cfg.device.qubit.pulses.pi_ge.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]})
print(f'*New pi_ge.gain\n', yaml_cfg.device.qubit.pulses.pi_ge.gain)
yaml_cfg.device.qubit.pulses.pi_ge.gain = yaml_cfg.device.qubit.pulses.pi_ge.gain.flatten().astype(int).tolist()
print(f'*New pi_ge.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.gain}')
print(results_Qi)

best 2 0.007626497366073614
Pi gain from amps data [dac units]: 4395
	Pi/2 gain from amps data [dac units]: 2197
Pi gain from avgi data [dac units]: 4654
	Pi/2 gain from avgi data [dac units]: 2327
Pi gain from avgq data [dac units]: 4448
	Pi/2 gain from avgq data [dac units]: 2224
*Set pi pulse sigma for qubit 1 to 0.025*
*Set pi pulse gain for qubit 1 to 4448*
*New pi_ge.gain
 [[1849 1914    0 1855]
 [2834 4448 2648 2631]
 [9155 9155 9155 9155]
 [ 599  599  599  599]]
*New pi_ge.gain flattened
[1849, 1914, 0, 1855, 2834, 4448, 2648, 2631, 9155, 9155, 9155, 9155, 599, 599, 599, 599]
{'Pi pulse gain': 4448}


In [26]:
if save_data: saved_files.append(amprabi.save_data())

## $T_1$

In [18]:
short_T1 = False
# short_T1 = True

In [19]:
t1 = meas.T1Experiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"t1_qubit{qubit_i}",
    config_file=config_path,
)
t1.cfg = AttrDict(deepcopy(yaml_cfg))

span = 50 if short_T1 else 400
npts = 151

t1.cfg.expt = dict(
    start=0, # wait time [us]
    step=span/npts,
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    qTest=qubit_i,
)
t1.cfg.device.readout.relax_delay = span # Wait time between experiments [us]
t1.go(analyze=False, display=False, progress=True, save=False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [20]:
# # TESTING
# # t1 = meas.T1Experiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_t1_qubit1.h5')
# t1.data = temp_data

t1.analyze(fit_log=False)

t1_fit, t1_fit_err = meas.fitting.get_best_fit(t1.data) #, fitter.expfunc)

t1.display(fit=True, fit_log=False)
t1_qubit = t1_fit[3]
yaml_cfg.device.readout.relax_delay[qubit_i] = np.round(4*t1_qubit)
print(f'*Set readout {qubit_i} relax delay to {yaml_cfg.device.readout.relax_delay[qubit_i]}*')
results_Qi.update({'T1':t1_qubit, 'T1 err':np.sqrt(t1_fit_err[3][3])})
print(f'qubit {qubit_i}', results_Qi)

best 0 87137.91192366534
Fit T1 amps [us]: 64.05955000251987
Fit T1 avgi [us]: 63.89865426749213
Fit T1 avgq [us]: 67.81473166501338
*Set readout 0 relax delay to 256.0*
qubit 0 {'T1': 64.05955000251987, 'T1 err': 1.119240285515182}


In [30]:
if save_data: saved_files.append(t1.save_data())

## $T_2$ Ramsey round 1

In [20]:
# yaml_cfg.device.qubit.f_ge[qubit_i] = 145.572034953271
qubit_hwhm = 2
short_T1 = False
# short_T1 = t1_qubit < 20
print('Short T1', short_T1)

Short T1 False


In [21]:
# ramsey_freq = max((min((np.abs(np.ceil(qubit_hwhm)), 3)), 2))
ramsey_freq = 3.0
print(f'Ramsey freq [MHz]: {ramsey_freq}')

Ramsey freq [MHz]: 3.0


In [22]:
t2r = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"ramsey_qubit{qubit_i}",
    config_file=config_path,
)
t2r.cfg = AttrDict(deepcopy(yaml_cfg))

# make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
t2r.cfg.expt = dict(
    start=0, # wait time tau [us]
    step=soc.cycles2us(8), # [us] 
    # step=soc.cycles2us(2) if short_T1 else soc.cycles2us(10), # [us] 
    expts=200,
    ramsey_freq=ramsey_freq, # [MHz]
    reps=reps,
    rounds=rounds,
    qTest=qubit_i,
    checkEF=False,
    qZZ=None,
)

try:
    t2r.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

Running Ramsey on Q0 


  0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
# # TESTING
# # t2r = meas.RamseyExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00000_ramsey_qubit2.h5')
# t2r.data = temp_data

# t2r.analyze(fit=True, fit_num_sin=2)
# t2r.display(fit=True, fit_num_sin=2)

t2r.analyze(fit=True)
t2r_fit, t2r_fit_err, t2r_adjust = meas.fitting.get_best_fit(t2r.data, get_best_data_params=['f_adjust_ramsey'])

old_qubit_freq = np.reshape(t2r.cfg.device.qubit.f_ge, (4,4))[qubit_i, qubit_i]
t2r.display(fit=True)
yaml_cfg.device.qubit.f_ge = np.reshape(yaml_cfg.device.qubit.f_ge, (4,4))
yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] = old_qubit_freq + t2r_adjust[0]
alt_qubit_freq = old_qubit_freq + t2r_adjust[1]
results_Qi.update({'T2R':t2r_fit[3], 'T2R err':np.sqrt(t2r_fit_err[3][3])})
results_Qi.update({'Qubit freq':yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i]}*')
print(f'Alt qubit freq is {alt_qubit_freq}')

print(f'*New f_ge\n', yaml_cfg.device.qubit.f_ge)
yaml_cfg.device.qubit.f_ge = yaml_cfg.device.qubit.f_ge.flatten().tolist()
print(f'*New f_ge flattened\n{yaml_cfg.device.qubit.f_ge}')

data = t2r.data
fit_axis = 'avgq'
xpts = data['xpts']
fourier = np.fft.fftshift(np.abs(np.fft.fft(data[fit_axis])))
freqs = np.fft.fftshift(np.fft.fftfreq(len(fourier), d=(xpts[1]-xpts[0])))
plt.plot(freqs, fourier)
ZZs = np.zeros((4,4))
reshaped_f_ge = np.reshape(t2r.cfg.device.qubit.f_ge, (4,4))
for row in range(4):
    ZZs[row,:] = reshaped_f_ge[row,:] - reshaped_f_ge[row,row]
for i in range(4):
    if (qubit_i+i)%4 == qubit_i: color = 'r'
    else: color = 'k'
    plt.axvline(t2r.cfg.expt.ramsey_freq - ZZs[qubit_i, (qubit_i+i)%4], color=color, linewidth=0.5, linestyle='--')
plt.xlabel('Frequency [MHz]')
plt.xlim(-6, 6)
# plt.ylim(0, 20000)
plt.title(f'Fourier Transform of Ramsey on Q{qubit_i} (Ramsey Freq: {t2r.cfg.expt.ramsey_freq})')
plt.show()

best 2 0.06298747911090018
Current pi pulse frequency: 4111.097866562327
Fit frequency from amps [MHz]: 2.9838925352135837 +/- 0.0015845702399833789
Suggested new pi pulse frequencies from fit amps [MHz]:
 	4111.113974027114
 	4105.113974027114
T2 Ramsey from fit amps [us]: 36.68291156637222 +/- 13.183563781308564
Current pi pulse frequency: 4111.097866562327
Fit frequency from I [MHz]: 2.9926650083224806 +/- 0.013064566157928049
Suggested new pi pulse frequency from fit I [MHz]:
 	4111.105201554004
 	4105.105201554004
T2 Ramsey from fit I [us]: 2.439038086143288 +/- 0.479549217717183
Fit frequency from Q [MHz]: 2.9842370848404647 +/- 0.0014295941257310605
Suggested new pi pulse frequencies from fit Q [MHz]:
 	4111.113629477487
 	4105.113629477487
T2 Ramsey from fit Q [us]: 31.91594097611952 +/- 9.000535283636879
*Set qubit 0 freq to 4111.113629477487*
Alt qubit freq is 4105.113629477487
*New f_ge
 [[4111.11362948 4105.83890476 4110.9581533  4110.54640785]
 [3442.87557211 3448.55302423

In [ ]:
if save_data: saved_files.append(t2r.save_data())

## Amplitude Rabi round 2

In [24]:
pi_len = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]
print('qubit', qubit_i, 'pi_len', pi_len)
# pi_len = 0.025

qubit 0 pi_len 0.02


In [25]:
amprabi_2 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi_2.cfg = AttrDict(deepcopy(yaml_cfg))

n_pulses = 1
span = min((32000, int(1.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i])))
npts = 151
    
amprabi_2.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    sigma_test=pi_len, # gaussian sigma for pulse length - overrides config [us]
    pulse_type='gauss',
    checkEF=False, 
    n_pulses=n_pulses,
    qTest=qubit_i,
    qZZ=None,
)
amprabi_2.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi on Q0 


  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# # TESTING
# # amprabi_2 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi_2.data = temp_data

amprabi_2.analyze(fit=True)

amprabi_2_fit, amprabi_2_fit_err = meas.fitting.get_best_fit(amprabi_2.data)

p = amprabi_2_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi_2.display(fit=False)
    print(f'WARNING: Fit pi pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi_2.display(fit=True)
yaml_cfg.device.qubit.pulses.pi_ge.sigma[qubit_i] = amprabi_2.cfg.expt.sigma_test

print(f'*Set pi pulse sigma for qubit {qubit_i} to {np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]}*')
yaml_cfg.device.qubit.pulses.pi_ge.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]})
print(f'*New pi_ge.gain\n', yaml_cfg.device.qubit.pulses.pi_ge.gain)
yaml_cfg.device.qubit.pulses.pi_ge.gain = yaml_cfg.device.qubit.pulses.pi_ge.gain.flatten().tolist()
print(f'*New pi_ge.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.gain}')
print(results_Qi)

best 0 0.022292039093864034
Pi gain from amps data [dac units]: 935
	Pi/2 gain from amps data [dac units]: 467
Pi gain from avgi data [dac units]: 935
	Pi/2 gain from avgi data [dac units]: 467
Pi gain from avgq data [dac units]: 936
	Pi/2 gain from avgq data [dac units]: 468
*Set pi pulse sigma for qubit 3 to 0.025*
*Set pi pulse gain for qubit 3 to 935*
*New pi_ge.gain
 [[ 1849  1914     0  1855]
 [ 2834  2504  2648  2631]
 [14489 14572 14736 14639]
 [  963   959   974   935]]
*New pi_ge.gain flattened
[1849, 1914, 0, 1855, 2834, 2504, 2648, 2631, 14489, 14572, 14736, 14639, 963, 959, 974, 935]
{'T1': 52.3580045090926, 'T1 err': 1.6646251872930162, 'T2R': 28.080322737694868, 'T2R err': 10.12437756373936, 'Qubit freq': 4381.110012654556, 'Pi pulse gain': 935}


In [38]:
if save_data: saved_files.append(amprabi_2.save_data())

## $T_2$ Ramsey round 2

Check that frequency adjustment helps.

In [26]:
t2r_2 = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"ramsey_qubit{qubit_i}",
    config_file=config_path,
)
t2r_2.cfg = AttrDict(deepcopy(yaml_cfg))

# make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
t2r_2.cfg.expt = dict(
    start=0, # wait time tau [us]
    # step=soc.cycles2us(2) if t2r_fit[3] < 3 else soc.cycles2us(60), # [us]
    step=soc.cycles2us(150), # [us]
    expts=151,
    ramsey_freq=ramsey_freq/15, # [MHz]
    reps=reps,
    rounds=rounds,
    checkEF=False,
    qTest=qubit_i,
    qZZ=None,
)
t2r_2.go(analyze=False, display=False, progress=True, save=False)

Running Ramsey on Q0 


  0%|          | 0/10 [00:00<?, ?it/s]

In [27]:
# TESTING
# t2r_2 = meas.RamseyExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00004_ramsey_qubit2.h5')
# t2r_2.data = temp_data
# t2r_2.cfg = AttrDict(attrs['config'])

redo_t2r = True

t2r_2.analyze(fit=True)

t2r_2_fit, t2r_2_fit_err, t2r_2_adjust = meas.fitting.get_best_fit(t2r_2.data, get_best_data_params=['f_adjust_ramsey'])

t2r_2.display(fit=True)
old_qubit_freq = np.reshape(t2r_2.cfg.device.qubit.f_ge, (4,4))[qubit_i, qubit_i]
if abs(t2r_2_adjust[0]) <= ramsey_freq/10:
# if abs(t2r_2_adjust[0]) <= abs(t2r_2_adjust[0]):
    redo_t2r = False
    yaml_cfg.device.qubit.f_ge = np.reshape(yaml_cfg.device.qubit.f_ge, (4,4))
    yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] = old_qubit_freq + t2r_2_adjust[0]
    results_Qi.update({'T2R':t2r_2_fit[3], 'T2R err':np.sqrt(t2r_2_fit_err[3][3])})
    results_Qi.update({'Qubit freq':yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
    print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i]}*')
    print(results_Qi)
    print(f'*New f_ge\n', yaml_cfg.device.qubit.f_ge)
    yaml_cfg.device.qubit.f_ge = yaml_cfg.device.qubit.f_ge.flatten().tolist()
    print(f'*New f_ge flattened\n{yaml_cfg.device.qubit.f_ge}')

else: # redo_t2r = True
    yaml_cfg.device.qubit.f_ge[qubit_i] = alt_qubit_freq 
    print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit.f_ge[qubit_i]}*')
print(f'Redo T2R: {redo_t2r}')

best 2 0.02192074858908997
Current pi pulse frequency: 4111.113629477487
Fit frequency from amps [MHz]: 0.20393123069817795 +/- 0.00025750104344985676
Suggested new pi pulse frequencies from fit amps [MHz]:
 	4111.109698246789
 	4110.709698246788
T2 Ramsey from fit amps [us]: 40.54835742698069 +/- 2.5688655555272253
Current pi pulse frequency: 4111.113629477487
Fit frequency from I [MHz]: 0.2028018168868628 +/- 0.0012709710316189794
Suggested new pi pulse frequency from fit I [MHz]:
 	4111.1108276606
 	4110.7108276606
T2 Ramsey from fit I [us]: 34.643315311645644 +/- 9.485646615179002
Fit frequency from Q [MHz]: 0.20387921070680948 +/- 0.0002286684534252612
Suggested new pi pulse frequencies from fit Q [MHz]:
 	4111.10975026678
 	4110.70975026678
T2 Ramsey from fit Q [us]: 40.909393879645755 +/- 2.3238675552340635
*Set qubit 0 freq to 4111.10975026678*
{'Res freq': 6809.964999999999, 'Res freq (yaml)': -440.03500000000037, 'Res kappa': 0.24998935111829518, 'T2R': 40.909393879645755, 'T

In [98]:
if save_data: saved_files.append(t2r_2.save_data())

## $\pi$, -$\pi$ experiment

In [130]:
use_robust_pulses = True

In [ ]:
piminuspi = meas.PiMinusPiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"PiMinusPiExpt{qubit_i}",
    config_file=config_path,
)

piminuspi.cfg = AttrDict(deepcopy(yaml_cfg))

span_f = 2.0
# npts_f = 50
npts_f = 30

if not use_robust_pulses:
    start_f = piminuspi.cfg.device.qubit.f_ge[qubit_i*4 + qubit_i]-span_f/2
    pulse_type = "gauss"
else:
    start_f=piminuspi.cfg.device.qubit.f_ge_robust[qubit_i*4 + qubit_i]-span_f/2,
    pulse_type = "robust"

piminuspi.cfg.expt = dict(
    start_N=0, # number gate repetitions
    step_N=4,
    expts_N=10,
    start_f=start_f,
    step_f=span_f/npts_f,
    expts_f=npts_f+1,
    reps=int(0.75*reps*rounds),
    # reps=10,
    loops=1,
    pulse_type=pulse_type,
    use_robust_pulses=use_robust_pulses,

    checkEF=False,
    qTest=qubit_i,
    qZZ=None,

    test_pi_half=False,

    post_process='threshold',
    singleshot_reps=15000,
)

# print(piminuspi.cfg)
import Pyro4.util
try:
    piminuspi.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 78.30666666666667 	 angle (deg): -96.83569591568404 	 threshold ge: 398.7212265605594


  0%|          | 0/10 [00:00<?, ?it/s]

In [132]:
# # TESTING
# temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00000_PiMinusPiExpt0.h5')
# piminuspi.data =  temp_data
# piminuspi.cfg = AttrDict(attrs['config'])


data = piminuspi.data
piminuspi.analyze(fit=True)
piminuspi.display(fit=True)

if not use_robust_pulses:
    freq_update = "f_ge"
else:
    freq_update = "f_ge_robust"
yaml_cfg.device.qubit[freq_update] = np.reshape(yaml_cfg.device.qubit[freq_update], (4,4))
yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i] = data["best_freq"]
results_Qi.update({'Qubit freq':yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i]}*')
print(results_Qi)
print(f'*New {freq_update}\n', yaml_cfg.device.qubit[freq_update])
yaml_cfg.device.qubit[freq_update] = yaml_cfg.device.qubit[freq_update].flatten().tolist()
print(f'*New {freq_update} flattened\n{yaml_cfg.device.qubit[freq_update]}')

Fit best freq 4111.8334058569635 which is -0.47307206386085454 away from old freq 4112.306477920824
*Set qubit 0 freq to 4111.8334058569635*
{'T1': 64.05955000251987, 'T1 err': 1.119240285515182, 'T2R': 18.248886781332068, 'T2R err': 0.7816516069106116, 'Qubit freq': 4111.8334058569635, 'T2E': 56.36959417286505, 'T2E err': 1.4283912853884384, 'Res freq': 6810.011666666666, 'Res freq (yaml)': -389.98833333333334, 'Res kappa': 0.2735895597906132, 'Res chi shift (e - g)': -0.8642384105960446, 'Pi/2 pulse gain': 7569}
*New f_ge_robust
 [[4111.83340586 4112.30647792 4112.30647792 4112.30647792]
 [3450.05408931 3450.05408931 3450.05408931 3450.05408931]
 [4763.941      4763.941      4763.941      4763.941     ]
 [4384.25034226 4384.25034226 4384.25034226 4384.25034226]]
*New f_ge_robust flattened
[4111.8334058569635, 4112.306477920824, 4112.306477920824, 4112.306477920824, 3450.0540893078382, 3450.0540893078382, 3450.0540893078382, 3450.0540893078382, 4763.941, 4763.941, 4763.941, 4763.941, 

In [ ]:
if save_data: piminuspi.save_data()

## GE error amplification

In [ ]:
npulsecalib = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptQ{qubit_i}",
    config_file=config_path,
)

npulsecalib.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=10,
    reps=1000,
    loops=20,
    pulse_type='gauss',
    checkEF=False, 
    qTest=qubit_i,
    qZZ=None,
    test_pi_half=False,
    error_amp=True,

    post_process='scale',
    singleshot_reps=15000,
)

# print(npulsecalib.cfg)
import Pyro4.util
try:
    npulsecalib.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib.data
npulsecalib.analyze(fit=True, scale=None)
npulsecalib.display(fit=True, scale=None)

# npulsecalib_fit, npulsecalib_fit_err = meas.fitting.get_best_fit(npulsecalib.data)
fit_axis = 'avgi'
npulsecalib_fit = data['fit_avgi']
npulsecalib_fit_err = data['fit_err_avgi']

angle_err = npulsecalib_fit[1]
amp_ratio = (180 - angle_err) / 180
new_amp_ge = np.reshape(npulsecalib.cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i] / amp_ratio

yaml_cfg.device.qubit.pulses.pi_ge.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i] = int(round(new_amp_ge))
print(f'*Set pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]})
print(f'*New pi_ge.gain\n', yaml_cfg.device.qubit.pulses.pi_ge.gain)
yaml_cfg.device.qubit.pulses.pi_ge.gain = yaml_cfg.device.qubit.pulses.pi_ge.gain.flatten().tolist()
print(f'*New pi_ge.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.gain}')
print(results_Qi)

In [ ]:
if save_data: npulsecalib.save_data()

## $T_2$ Echo

In [26]:
# yaml_cfg.device.qubit.f_ge[qubit_i] = 145.572034953271
qubit_hwhm = 2
short_T1 = False
# short_T1 = True
# short_T1 = yaml_cfg.device.qubit.T1[qubit_i] < 20
print('Short T1', short_T1)

Short T1 False


In [27]:
ramsey_freq = max((min((np.abs(np.ceil(qubit_hwhm)), 3)), 2))
# ramsey_freq = 1
print(f'Ramsey freq [MHz]: {ramsey_freq}')

Ramsey freq [MHz]: 2.0


In [28]:
t2e = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"echo_qubit{qubit_i}",
    config_file=config_path,
)
t2e.cfg = AttrDict(deepcopy(yaml_cfg))

t2e.cfg.expt = dict(
    start=0, # total wait time b/w the two pi/2 pulses [us]
    # step=soc.cycles2us(80), # make sure nyquist freq = 0.5*(1/step) > ramsey (signal) freq!
    step=soc.cycles2us(30) if short_T1 else soc.cycles2us(150), # [us] 
    expts=201,
    ramsey_freq=max((ramsey_freq/15, 0.100)), # frequency by which to advance phase [MHz]
    num_pi=1, # number of pi pulses
    echo_type='cpmg', # cp or cpmg
    reps=reps,
    # reps=10,
    rounds=rounds,
    qTest=qubit_i,
    qZZ=None,
    checkEF=False,
)
t2e.go(analyze=False, display=False, progress=True, save=False)

Running Ramsey Echo on Q0 


  0%|          | 0/10 [00:00<?, ?it/s]

In [29]:
# # TESTING
# # t2e = meas.RamseyEchoExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00000_echo_qubit1.h5')
# t2e.data = temp_data

print(f'Qubit {qubit_i}')
t2e.analyze(fit=True)

t2e_fit, t2e_fit_err = meas.fitting.get_best_fit(t2e.data)

if not isinstance(t2e_fit, (list, np.ndarray)):
    t2e.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

t2e.display(fit=True)
results_Qi.update({'T2E':t2e_fit[3], 'T2E err':np.sqrt(t2e_fit_err[3][3])})
print(results_Qi)
# print('Results updated.')

Qubit 0
best 0 0.009515671840636859
Current pi pulse frequency: 4111.122260451728
Fit frequency from amps [MHz]: 0.13321301485526182 +/- 7.214999297595147e-05
Suggested new pi pulse frequencies from fit amps [MHz]:
 	4111.122380770206
 	4110.855714103539
T2 Ramsey from fit amps [us]: 56.36959417286505 +/- 1.4283912853884384
Current pi pulse frequency: 4111.122260451728
Fit frequency from I [MHz]: 0.13315630305524895 +/- 7.78367439779741e-05
Suggested new pi pulse frequency from fit I [MHz]:
 	4111.122437482006
 	4110.855770815339
T2 Ramsey from fit I [us]: 56.049642889050325 +/- 1.523004844518435
Fit frequency from Q [MHz]: 0.13330409486482314 +/- 8.258644157305862e-05
Suggested new pi pulse frequencies from fit Q [MHz]:
 	4111.122289690196
 	4110.85562302353
T2 Ramsey from fit Q [us]: 56.803008426451235 +/- 1.6613331354971312
{'T1': 64.05955000251987, 'T1 err': 1.119240285515182, 'T2R': 18.248886781332068, 'T2R err': 0.7816516069106116, 'Qubit freq': 4111.122260451728, 'T2E': 56.36959

In [46]:
if save_data: saved_files.append(t2e.save_data())

## (Divide len) Calibrate $\pi/2$ pulse

### Amplitude rabi for $\pi/2$ pulse (divide len) (use this for protocol)

In [35]:
divide_len = True

In [36]:
pi_len = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]
# pi_len = 0.008
print('qubit', qubit_i, 'pi_len', pi_len)

qubit 1 pi_len 0.025


In [37]:
amprabi_3 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi_3.cfg = AttrDict(deepcopy(yaml_cfg))

span = min((32000, int(1.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i])))
npts = 151
    
amprabi_3.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    sigma_test=pi_len, # gaussian sigma for pulse length - overrides config [us]
    pulse_type='gauss',
    checkEF=False, 
    n_pulses=1,
    test_pi_half=True,
    qTest=qubit_i,
    qZZ=None,
)
amprabi_3.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi on Q1 
Calibrating half pi gain (divide length) when pi len is 0.025


  0%|          | 0/10 [00:00<?, ?it/s]

In [38]:
# # TESTING
# # amprabi_3 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi_3.data = temp_data

# amprabi_3.display(fit=False)

amprabi_3.analyze(fit=True)

amprabi_3_fit, amprabi_3_fit_err = meas.fitting.get_best_fit(amprabi_3.data)

p = amprabi_3_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi_3.display(fit=False)
    print(f'WARNING: Fit pi/2 pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi_3.display(fit=True)

yaml_cfg.device.qubit.pulses.pi_ge.half_gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.half_gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.half_gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.half_gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.half_gain[qubit_i, qubit_i]})
print(f'*New pi_ge.half_gain\n', yaml_cfg.device.qubit.pulses.pi_ge.half_gain)
yaml_cfg.device.qubit.pulses.pi_ge.half_gain = yaml_cfg.device.qubit.pulses.pi_ge.half_gain.flatten().astype(int).tolist()
print(f'*New pi_ge.half_gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.half_gain}')
print(results_Qi)

best 1 0.01354141233622637
Pi gain from amps data [dac units]: 2228
	Pi/2 gain from amps data [dac units]: 1114
Pi gain from avgi data [dac units]: 2213
	Pi/2 gain from avgi data [dac units]: 1106
Pi gain from avgq data [dac units]: 2198
	Pi/2 gain from avgq data [dac units]: 1099
*Set pi/2 pulse gain for qubit 1 to 2213*
*New pi_ge.half_gain
 [[ 1856  1835  1873  1867]
 [ 2382  2213  2248  2289]
 [14616 14662 14508 14535]
 [  819   819   816   819]]
*New pi_ge.half_gain flattened
[1856, 1835, 1873, 1867, 2382, 2213, 2248, 2289, 14616, 14662, 14508, 14535, 819, 819, 816, 819]
{'T1': 73.39553692251104, 'T1 err': 3.489431034704649, 'Pi/2 pulse gain': 2213}


In [ ]:
if save_data: saved_files.append(amprabi_3.save_data())

### $\pi/2$, -$\pi/2$ experiment

In [153]:
use_robust_pulses = True

In [154]:
sweep_phase = True

In [163]:
piminuspi = meas.PiMinusPiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"PiMinusPiExpt{qubit_i}",
    config_file=config_path,
)

piminuspi.cfg = AttrDict(deepcopy(yaml_cfg))

span_f = 5.0
npts_f = 30
# span_f = 2.0
# npts_f = 10

# span_f = 0.0
# npts_f = 1

if not use_robust_pulses:
    start_f = piminuspi.cfg.device.qubit.f_ge[qubit_i*4 + qubit_i]-span_f/2
    pulse_type = "gauss"
else:
    start_f=piminuspi.cfg.device.qubit.f_ge_robust[qubit_i*4 + qubit_i]-span_f/2,
    pulse_type = "robust"

if sweep_phase:
    start_f = -span_f/2

piminuspi.cfg.expt = dict(
    start_N=0, # number gate repetitions
    step_N=1,
    expts_N=10,
    # step_N=4,
    # expts_N=4,
    start_f=start_f,
    step_f=span_f/npts_f,
    expts_f=npts_f+1,
    # reps=int(0.75*reps*rounds),
    # reps=1000,
    reps=500,
    # reps=10,
    loops=1,
    pulse_type=pulse_type,
    use_robust_pulses=use_robust_pulses,

    checkEF=False,
    qTest=qubit_i,
    qZZ=None,

    # use_Y=True,

    test_pi_half=True,

    post_process='threshold',
    singleshot_reps=15000,

    sweep_phase=sweep_phase,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

# print(piminuspi.cfg)
import Pyro4.util
try:
    piminuspi.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 80.90594706313755 	 angle (deg): 0.0 	 threshold ge: 210.11307517492918


  0%|          | 0/10 [00:00<?, ?it/s]

0 0 0 phase 0
0 1 0 phase -198.0
0 0 0 phase 0
0 1 0 phase -196.8
0 0 0 phase 0
0 1 0 phase -195.6
0 0 0 phase 0
0 1 0 phase -194.4
0 0 0 phase 0
0 1 0 phase -193.2
0 0 0 phase 0
0 1 0 phase -192.0
0 0 0 phase 0
0 1 0 phase -190.8
0 0 0 phase 0
0 1 0 phase -189.6
0 0 0 phase 0
0 1 0 phase -188.4
0 0 0 phase 0
0 1 0 phase -187.2
0 0 0 phase 0
0 1 0 phase -186.0
0 0 0 phase 0
0 1 0 phase -184.8
0 0 0 phase 0
0 1 0 phase -183.6
0 0 0 phase 0
0 1 0 phase -182.4
0 0 0 phase 0
0 1 0 phase -181.2
0 0 0 phase 0
0 1 0 phase -180
0 0 0 phase 0
0 1 0 phase -178.8
0 0 0 phase 0
0 1 0 phase -177.6
0 0 0 phase 0
0 1 0 phase -176.4
0 0 0 phase 0
0 1 0 phase -175.2
0 0 0 phase 0
0 1 0 phase -174.0
0 0 0 phase 0
0 1 0 phase -172.8
0 0 0 phase 0
0 1 0 phase -171.6
0 0 0 phase 0
0 1 0 phase -170.4
0 0 0 phase 0
0 1 0 phase -169.2
0 0 0 phase 0
0 1 0 phase -168.0
0 0 0 phase 0
0 1 0 phase -166.8
0 0 0 phase 0
0 1 0 phase -165.6
0 0 0 phase 0
0 1 0 phase -164.4
0 0 0 phase 0
0 1 0 phase -163.2
0 0 0 phase 

In [164]:
# # TESTING
# temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00000_PiMinusPiExpt0.h5')
# piminuspi.data =  temp_data
# piminuspi.cfg = AttrDict(attrs['config'])


data = piminuspi.data
piminuspi.analyze(fit=True)
piminuspi.display(fit=True)

if sweep_phase:
    best_phase = data["best_freq"] * 2 * np.pi * piminuspi.cfg.expt.sigma_test * 180 / np.pi
    if piminuspi.cfg.expt.pulse_type == "gauss" or not piminuspi.cfg.expt.use_robust_pulses:
        best_phase = 4 * cfg.expt.Z_phase  # gaussian pulse is 4*listed length
    print(f"Best virtual Z (deg)", best_phase)

# if not use_robust_pulses:
#     freq_update = "f_ge"
# else:
#     freq_update = "f_ge_robust"
# yaml_cfg.device.qubit[freq_update] = np.reshape(yaml_cfg.device.qubit[freq_update], (4,4))
# yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i] = data["best_freq"]
# results_Qi.update({'Qubit freq':yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
# print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i]}*')
# print(results_Qi)
# print(f'*New {freq_update}\n', yaml_cfg.device.qubit[freq_update])
# yaml_cfg.device.qubit[freq_update] = yaml_cfg.device.qubit[freq_update].flatten().tolist()
# print(f'*New {freq_update} flattened\n{yaml_cfg.device.qubit[freq_update]}')

Fit best freq 0.008721164655621142 which is 0.008721164655621142 away from old freq 0
Best virtual Z (deg) 0.06279238552047223


In [ ]:
if save_data: piminuspi.save_data()

### GE $\pi/2$ error amplification (divide len)

In [196]:
divide_len = True

In [197]:
npulsecalib_gehalf = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptQ{qubit_i}",
    config_file=config_path,
)

if not use_robust_pulses:
    pulse_type = "gauss"
else:
    pulse_type = "robust"

npulsecalib_gehalf.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib_gehalf.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=20,
    reps=1000,
    loops=20,
    pulse_type=pulse_type,
    test_pi_half=True,
    divide_len=divide_len,
    error_amp=True,
    use_robust_pulses=use_robust_pulses,
    use_Y=False,

    checkEF=False,
    qTest=qubit_i,
    qZZ=None,

    post_process='scale',
    singleshot_reps=15000,
)

# print(npulsecalib_gehalf.cfg)
try:
    npulsecalib_gehalf.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (3)
ge fidelity (%): 81.28 	 angle (deg): 34.05335509852408 	 threshold ge: -666.1487237776781


  0%|          | 0/20 [00:00<?, ?it/s]

 s:\Connie\experiments\qram_tprocv1_expts\experiments\clifford_averager_program.py: 1742

In [202]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib_gehalf.data
npulsecalib_gehalf.analyze(fit=True, scale=None)
npulsecalib_gehalf.display(fit=True, scale=None)

# npulsecalib_gehalf_fit, npulsecalib_gehalf_fit_err = meas.fitting.get_best_fit(npulsecalib_gehalf.data)
fit_axis = 'avgi'
npulsecalib_gehalf_fit = data['fit_avgi']
npulsecalib_gehalf_fit_err = data['fit_err_avgi']

angle_err = npulsecalib_gehalf_fit[1]
amp_ratio = (90 - angle_err)/90
new_amp_half_ge = npulsecalib_gehalf.cfg.expt.gain / amp_ratio

if npulsecalib_gehalf.cfg.expt.use_robust_pulses:
    yaml_update = ["pihalf_ge_robust", "gain"]
else:
    yaml_update = ["pi_ge", "half_gain"]

yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]] = np.reshape(yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]], (4,4))
yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]][qubit_i, qubit_i] = int(round(new_amp_half_ge))
print(f'*Set pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]][qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 pulse gain':yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]][qubit_i, qubit_i]})
print(f'*New {yaml_update[0]}.{yaml_update[1]}\n', yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]])
yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]] = yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]].flatten().astype(int).tolist()
print(f'*New {yaml_update[0]}.{yaml_update[1]} flattened\n{yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]]}')
print(results_Qi)

fitparams [None, 0.017453292519943295]
From amps: adjust amplitude to 3595 / 1.0526538911009298 = 3415.1776100310894
From avgi: adjust amplitude to 3595 / 1.004050416624822 = 3580.4974934274874
From avgq: adjust amplitude to 3595 / 0.947346260195975 = 3794.8109904991993
*Set pi/2 pulse gain for qubit 3 to 3580*
*New pihalf_ge_robust.gain
 [[ 7569  7569  7569  7569]
 [ 4982  4982  4982  4982]
 [31856 31856 31856 31856]
 [ 3595  3595  3595  3580]]
*New pihalf_ge_robust.gain flattened
[7569, 7569, 7569, 7569, 4982, 4982, 4982, 4982, 31856, 31856, 31856, 31856, 3595, 3595, 3595, 3580]
{'Res freq': 6971.126666666667, 'Res freq (yaml)': -228.87333333333333, 'Res kappa': 0.21493746441942563, 'Res chi shift (e - g)': -0.8046357615893953, 'Qubit freq': 4383.7527021237, 'Pi/2 pulse gain': 3580}


In [ ]:
if save_data: npulsecalib_gehalf.save_data()

## (Divide gain) Calibrate $\pi/2$ pulse

### Amplitude rabi for $\pi/2$ pulse (divide gain) (use this for RB)

In [ ]:
divide_len = False

In [ ]:
pi_len = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]
# pi_len = 0.008
print('qubit', qubit_i, 'pi_len', pi_len)

In [ ]:
amprabi_3 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi_3.cfg = AttrDict(deepcopy(yaml_cfg))

span = min((32000, int(1.5*0.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i])))
npts = 151
    
amprabi_3.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    sigma_test=pi_len, # gaussian sigma for pulse length - overrides config [us]
    pulse_type='gauss',
    checkEF=False, 
    n_pulses=1,
    test_pi_half=True,
    divide_len=divide_len,
    qTest=qubit_i,
    qZZ=None,
)
amprabi_3.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
# # TESTING
# # amprabi_3 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi_3.data = temp_data

amprabi_3.analyze(fit=True)

amprabi_3_fit, amprabi_3_fit_err = meas.fitting.get_best_fit(amprabi_3.data)

p = amprabi_3_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi_3.display(fit=False)
    print(f'WARNING: Fit pi/2 pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi_3.display(fit=True)

yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i]})
print(f'*New pi_ge.half_gain_pi_sigma\n', yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma)
yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma = yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma.flatten().astype(int).tolist()
print(f'*New pi_ge.half_gain_pi_sigma flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma}')
print(results_Qi)

In [ ]:
if save_data: saved_files.append(amprabi_3.save_data())

### GE $\pi/2$ error amplification (divide gain)

In [ ]:
divide_len = False

In [ ]:
npulsecalib_gehalf = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptQ{qubit_i}",
    config_file=config_path,
)

npulsecalib_gehalf.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib_gehalf.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=10,
    reps=1000,
    loops=20,
    pulse_type='gauss',
    checkEF=False, 
    qTest=qubit_i,
    qZZ=None,
    test_pi_half=True,
    divide_len=divide_len,
    error_amp=True,

    post_process='scale',
    singleshot_reps=15000,
)

print(npulsecalib_gehalf.cfg)
import Pyro4.util
try:
    npulsecalib_gehalf.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib_gehalf.data
npulsecalib_gehalf.analyze(fit=True, scale=None)
npulsecalib_gehalf.display(fit=True, scale=None)

# npulsecalib_gehalf_fit, npulsecalib_gehalf_fit_err = meas.fitting.get_best_fit(npulsecalib_gehalf.data)
fit_axis = 'avgi'
npulsecalib_gehalf_fit = data['fit_avgi']
npulsecalib_gehalf_fit_err = data['fit_err_avgi']

angle_err = npulsecalib_gehalf_fit[1]
amp_ratio = (90 - angle_err)/90
new_amp_half_ge = np.reshape(npulsecalib_gehalf.cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma, (4,4))[qubit_i, qubit_i] / amp_ratio

yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i] = int(round(new_amp_half_ge))
print(f'*Set pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i]})
print(f'*New pi_ge.half_gain_pi_sigma\n', yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma)
yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma = yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma.flatten().astype(int).tolist()
print(f'*New pi_ge.half_gain_pi_sigma flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma}')
print(results_Qi)

In [ ]:
if save_data: npulsecalib_gehalf.save_data()

## Resonator chi shift (g/e)

In [51]:
rspec_chi = meas.ResonatorSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"resonator_spectroscopy_chi_qubit{qubit_i}",
    config_file=config_path,
)
rspec_chi.cfg = AttrDict(deepcopy(yaml_cfg))

span = 3 # MHz
npts = 151
rspec_chi.cfg.expt = dict(
    start=rspec_chi.cfg.device.readout.frequency[qubit_i]-span/2, # MHz
    # start=rspec_chi.cfg.device.readout.frequency[qubit_i]-rspec_chi.cfg.device.readout.lo_sideband[qubit_i]*span, # MHz
    step=span/npts,
    expts=npts,
    reps=reps*rounds,
    pulse_e=True, # add ge pi pulse prior to measurement
    pulse_f=False, # add ef pi pulse prior to measurement
    qTest=qubit_i,
    qZZ=None,
)

rspec_chi.cfg.device.readout.gain[qubit_i] /= 2
rspec_chi.cfg.device.readout.readout_length[qubit_i] *= 2

# rspec_chi.cfg.device.readout.relax_delay = 100 # Wait time between experiments [us]
rspec_chi.go(analyze=False, display=False, progress=True, save=False)

  0%|          | 0/151 [00:00<?, ?it/s]

In [52]:
# # TESTING
# # rspec_chi = meas.ResonatorSpectroscopyExperiment(config_file=config_path,) 
# temp_data, attrs = prev_data(expt_path, '00001_resonator_spectroscopy_qubit2.h5')
# rspec_chi.data = temp_data

rspec_chi.data = rspec_chi.analyze(fit=True)
rspec_chi_fit = rspec_chi.data['fit']
if not isinstance(rspec_chi_fit, (list, np.ndarray)):
    rspec_chi.display(fit=False)
    assert False, 'Fit failed, please manually fix!'
rspec_chi.display(fit=True)
# res_freq_e = rspec_chi_fit[0] - yaml_cfg.hw.soc.dacs.readout.mixer_freq[qubit_i]
res_freq_e = rspec_chi.data['xpts'][np.argmin(rspec_chi.data['amps'])]
# res_freq_e = rspec_chi.data['xpts'][np.argmax(rspec_chi.data['amps'])]
print('Res freq e', res_freq_e)
res_freq_g = yaml_cfg.device.readout.frequency[qubit_i]
print('Res freq g', res_freq_g)
# res_freq_e = -83
# results_Qi.update({'Res chi shift (e - g)': yaml_cfg.device.readout.lo_sideband[qubit_i]*(res_freq_e - yaml_cfg.device.readout.frequency[qubit_i])})
chi_shift_eg = yaml_cfg.device.readout.lo_sideband[qubit_i]*(res_freq_e - res_freq_g)
print('Chi shift (e-g):', chi_shift_eg)
results_Qi.update({'Res chi shift (e - g)': chi_shift_eg})
# res_freq_e = 6953.5
# results_Qi.update({'Res chi shift (e - g)': res_freq_e - (5925 + yaml_cfg.device.readout.lo_sideband[qubit_i]*(yaml_cfg.hw.soc.dacs.readout.mixer_freq + yaml_cfg.device.readout.frequency[qubit_i]))})
print(f'qubit {qubit_i}', results_Qi)

electrical delay phase correction (no mux support) (deg) 2.304607827865621
param guess [6971.083399558499, 34855.41699779249, 34855.41699779249, 0, -0.020229080273269678, 0.9321716156826464]
param bounds ([6969.613200883003, 0, 0, -6.283185307179586, -0.030343620409904515, -inf], [6972.553598233996, 1000000000.0, 100000000.0, 6.283185307179586, 0.030343620409904515, 1.0327736012061464])
popt [ 6.97032392e+03  5.13736337e+04  1.14974762e+05  4.56843738e-01
 -5.83184099e-03 -3.35698346e-02]

Freq with minimum transmission: -279.67156732891794
Freq with maximum transmission: -280.12852097130207
From fit:
	f0: 6970.323917858714
	Qi: 51373.633657959705 	 kappa_i/2pi: 0.13567901317369147
	Qe: 114974.76205806932 	 kappa_e/2pi: 0.06062481707366589
	Q0: 35507.83450875915
	kappa [MHz]: 0.19630383024735734
	phi [radians]: 0.45684373822349367
baseline 93.51404473612233
Res freq e -279.6516997792491
Res freq g -278.9066666666663
Chi shift (e-g): -0.7450331125828029
qubit 3 {'Res freq': 6971.0933333

In [49]:
if save_data: saved_files.append(rspec_chi.save_data())

# Qubit ef spectroscopy

## Pulse Probe EF spectroscopy

In [32]:
qEFspec = meas.PulseProbeSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"qubit_EF_spectroscopy_qubit{qubit_i}",
    config_file=config_path,
)
qEFspec.cfg = AttrDict(deepcopy(yaml_cfg))

span = 50
npts = 250

qEFspec.cfg.expt = dict(
    start=np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))[qubit_i,qubit_i]-0.5*span, # resonator frequency to be mixed up [MHz]
    step=span/npts, # min step ~1 Hz
    expts=npts, # Number of experiments stepping from start
    reps=reps, # Number of averages per point
    rounds=rounds, # Number of start to finish sweeps to average over
    length=1, # ef probe constant pulse length [us]
    gain=int(np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i]/50), # ef pulse gain
    # gain=5000,
    pulse_type='gauss',
    qTest=qubit_i,
    qZZ=None,
    checkEF=True,
)
# qEFspec.cfg.device.readout.relax_delay = qEFspec.cfg.device.qubit.T1[qubit_i] # Wait time between experiments [us]
qEFspec.go(analyze=False, display=False, progress=True, save=False)

Running pulse probe EF on Q3 


  0%|          | 0/10 [00:00<?, ?it/s]

In [33]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = qEFspec.analyze(data=temp_data, fit=True)

best_signs = [1, 0, 0]
best_fit_err_i = np.inf
best_fit_err_q = np.inf
for sign in [1, -1]:
    try: 
        qEFspec.analyze(fit=True, signs=[1, sign, sign])
        fit_err_i = qEFspec.data['fit_err_avgi'][2][2]
        fit_err_q = qEFspec.data['fit_err_avgq'][2][2]
        if fit_err_i < best_fit_err_i:
            best_signs[1] = sign
            best_fit_err_i = fit_err_i
        if fit_err_q < best_fit_err_q:
            best_signs[2] = sign
            best_fit_err_q = fit_err_q
    except: continue
if best_fit_err_i == np.inf and best_fit_err_q == np.inf:
    qEFspec.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

qEFspec.analyze(fit=True, signs=best_signs)
qEFspec.display(fit=True, signs=best_signs)

qEFspec_fit, qEFspec_fit_err = meas.fitting.get_best_fit(qEFspec.data)
yaml_cfg.device.qubit.f_ef = np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))
yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] = qEFspec_fit[2]
print(f'*Set qubit {qubit_i} f_ef to {yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i]}*')
results_Qi.update({'Qubit EF freq':yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
yaml_cfg.device.qubit.f_ef = yaml_cfg.device.qubit.f_ef.flatten().tolist()
print(results_Qi)

Found peak in amps at [MHz] 4208.640414461112, HWHM 1.769695788060476
Found peak in I at [MHz] 4208.6412131041925, HWHM 1.7779817638956872
Found peak in Q at [MHz] 4208.4474152777675, HWHM 1.3649681847859378
best 1 0.042699123985299584
*Set qubit 3 f_ef to 4208.6412131041925*
{'T1': 52.3580045090926, 'T1 err': 1.6646251872930162, 'T2R': 25.602583027942746, 'T2R err': 1.5551254720790846, 'Qubit freq': 4381.109690194443, 'Pi pulse gain': 935, 'T2E': 27.649964593090505, 'T2E err': 1.1781089735321566, 'Res chi shift (e - g)': -0.009933774834678388, 'Qubit EF freq': 4208.6412131041925}


In [107]:
if save_data: saved_files.append(qEFspec.save_data())

## Pulse Probe EF spectroscopy round 2

In [34]:
qEFspec2 = meas.PulseProbeSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"qubit_EF_spectroscopy_qubit{qubit_i}",
    config_file=config_path,
)
qEFspec2.cfg = AttrDict(deepcopy(yaml_cfg))

span = 15
npts = 250

qEFspec2.cfg.expt = dict(
    start=np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))[qubit_i,qubit_i]-0.5*span, # resonator frequency to be mixed up [MHz]
    step=span/npts, # min step ~1 Hz
    expts=npts, # Number of experiments stepping from start
    reps=reps, # Number of averages per point
    rounds=rounds, # Number of start to finish sweeps to average over
    length=1, # ef probe constant pulse length [us]
    gain=int(np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i]/80), # ef pulse gain
    pulse_type='gauss',
    qTest=qubit_i,
    qZZ=None,
    checkEF=True,
)
qEFspec2.go(analyze=False, display=False, progress=True, save=False)

Running pulse probe EF on Q3 


  0%|          | 0/10 [00:00<?, ?it/s]

In [35]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = qEFspec2.analyze(data=temp_data, fit=True)

best_signs = [1, 0, 0]
best_fit_err_i = np.inf
best_fit_err_q = np.inf
for sign in [1, -1]:
    qEFspec2.analyze(fit=True, signs=[1, sign, sign])
    fit_err_i = qEFspec2.data['fit_err_avgi'][2][2]
    fit_err_q = qEFspec2.data['fit_err_avgq'][2][2]
    if fit_err_i < best_fit_err_i:
        best_signs[1] = sign
        best_fit_err_i = fit_err_i
    if fit_err_q < best_fit_err_q:
        best_signs[2] = sign
        best_fit_err_q = fit_err_q
    # except: continue
if best_fit_err_i == np.inf and best_fit_err_q == np.inf:
    qEFspec2.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

qEFspec2.analyze(fit=True, signs=best_signs)
qEFspec2.display(fit=True, signs=best_signs)

qEFspec2_fit, qEFspec2_fit_err = meas.fitting.get_best_fit(qEFspec2.data)
yaml_cfg.device.qubit.f_ef = np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))
yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] = qEFspec2_fit[2]
qubit_hwhm = qEFspec2_fit[3]
print(f'*Set qubit {qubit_i} f_ef to {yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i]}*')
results_Qi.update({'Qubit EF freq':yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
yaml_cfg.device.qubit.f_ef = yaml_cfg.device.qubit.f_ef.flatten().tolist()
print(results_Qi)

Found peak in amps at [MHz] 4208.845439952777, HWHM 0.7411164792727137
Found peak in I at [MHz] 4208.845449298409, HWHM 0.7461517092695102
Found peak in Q at [MHz] 4208.805250551362, HWHM 0.5165292240166288
best 0 0.03135604252512325
*Set qubit 3 f_ef to 4208.845439952777*
{'T1': 52.3580045090926, 'T1 err': 1.6646251872930162, 'T2R': 25.602583027942746, 'T2R err': 1.5551254720790846, 'Qubit freq': 4381.109690194443, 'Pi pulse gain': 935, 'T2E': 27.649964593090505, 'T2E err': 1.1781089735321566, 'Res chi shift (e - g)': -0.009933774834678388, 'Qubit EF freq': 4208.845439952777}


In [110]:
if save_data: saved_files.append(qEFspec2.save_data())

## Amplitude EF Rabi

The EF pi pulse amplitude should be stored in <code>cfg.device.qubit.pulses.pi_ef.gain</code>.

In [16]:
pi_lenEF = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]
# pi_lenEF = 0.025
print('qubit', qubit_i, 'pi_lenEF', pi_lenEF)

qubit 1 pi_lenEF 0.025


In [17]:
amprabiEF = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix="amp_rabi_EF"+f"_qubit{qubit_i}",
    config_file=config_path,
)

amprabiEF.cfg = AttrDict(deepcopy(yaml_cfg))

span = min((32000, int(1.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i])))
npts = 101

amprabiEF.cfg.expt = dict(
    start=1, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps*rounds,
    rounds=1,
    sigma_test=pi_lenEF, # gaussian sigma for pulse length - default from cfg [us]
    pulse_type='gauss',
    checkEF=True,
    qTest=qubit_i,
    qZZ=None,

    readout_cool=False,
    # n_init_readout=1,
    # n_trig=2,
    # init_read_wait_us=5,
    # singleshot_reps=20000,
    # use_gf_readout=True,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

# print(amprabiEF.cfg)
amprabiEF.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi EF on Q1 


  0%|          | 0/101000 [00:00<?, ?it/s]

(4, 101, 1000)


In [18]:
# TESTING
# amprabiEF = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_EF_qubit1.h5')
# amprabiEF.data = temp_data

# amprabiEF.analyze(fit=False)
# amprabiEF.display(fit=False)

amprabiEF.analyze(fit=True, post_select=False)

amprabiEF_fit, amprabiEF_fit_err = meas.fitting.get_best_fit(amprabiEF.data)

p = amprabiEF_fit

if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabiEF.display(fit=False)
    print('WARNING: Fit pi pulse gain is long, please double check!')
    pi_gain = 30000
else: amprabiEF.display(fit=True)


best 0 0.04915627151792078
Pi gain from amps data [dac units]: 3332
	Pi/2 gain from amps data [dac units]: 1666
Pi gain from avgi data [dac units]: 5356
	Pi/2 gain from avgi data [dac units]: 2678
Pi gain from avgq data [dac units]: 3471
	Pi/2 gain from avgq data [dac units]: 1735


In [19]:
print(f'*Set EF pi pulse sigma for qubit {qubit_i} to {np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]}*')
yaml_cfg.device.qubit.pulses.pi_ef.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set EF pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]})
print(f'*New pi_ef.gain\n', yaml_cfg.device.qubit.pulses.pi_ef.gain)
yaml_cfg.device.qubit.pulses.pi_ef.gain = yaml_cfg.device.qubit.pulses.pi_ef.gain.flatten().astype(int).tolist()
print(f'*New pi_ef.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.gain}')
print(results_Qi)

*Set EF pi pulse sigma for qubit 1 to 0.025*
*Set EF pi pulse gain for qubit 1 to 3332*
*New pi_ef.gain
 [[ 1280     0     0     0]
 [    0  3332     0     0]
 [10000 10079 10015     0]
 [  300   300     0   300]]
*New pi_ef.gain flattened
[1280, 0, 0, 0, 0, 3332, 0, 0, 10000, 10079, 10015, 0, 300, 300, 0, 300]
{'Pi EF pulse gain': 3332}


In [115]:
if save_data: saved_files.append(amprabiEF.save_data())

## Length EF Rabi

In [ ]:
lengthrabiEF = meas.LengthRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"length_rabiEF_qubit{qubit_i}",
    config_file=config_path,
)
lengthrabiEF.cfg = AttrDict(deepcopy(yaml_cfg))

lengthrabiEF.cfg.expt = dict(
    start=soc.cycles2us(0), # pulse length (sigma) [us]
    step=soc.cycles2us(1), # [us]
    expts=121,
    reps=reps*rounds,
    pulse_type='gauss',
    checkEF=True, 
    qTest=qubit_i,
    qZZ=None,
)
# lengthrabiEF.cfg.device.readout.relax_delay = 50 # Wait time between experiments [us]

import Pyro4.util
try:
    lengthrabiEF.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
# # TESTING
# # lengthrabiEF = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# lengthrabiEF.data = temp_data

lengthrabiEF.analyze(fit=True, fit_func='sin')

lengthrabiEF_fit, lengthrabiEF_fit_err = meas.fitting.get_best_fit(lengthrabiEF.data) #, fitfunc=fitter.decaysin)

lengthrabiEF.display(fit=False, fit_func='sin')
# p = lengthrabiEF_fit
# if p[2] > 180: p[2] = p[2] - 360
# elif p[2] < -180: p[2] = p[2] + 360
# if p[2] < 0: pi_lenEF = (1/2 - p[2]/180)/2/p[1]
# else: pi_lenEF = (3/2 - p[2]/180)/2/p[1]
# pi_lenEF = np.max((0.9*pi_lenEF, 0.015)) # since we run the length rabi at a < max gain

# lengthrabiEF_decay = p[3]

# print(f'Will use sigma {pi_lenEF} for amplitude rabi EF')

In [ ]:
if save_data: saved_files.append(lengthrabiEF.save_data())

## Qubit temperature

In [33]:
amprabiEF2 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix="amp_rabi_EF"+f"_qubit{qubit_i}",
    config_file=config_path,
)

amprabiEF2.cfg = AttrDict(deepcopy(yaml_cfg))

reps_temp=500
rounds_temp=reps*rounds*100//500

span = np.reshape(amprabiEF2.cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i]
npts = 4
# reps_temp = 10
# rounds_temp = 10
print('reps', reps_temp, 'rounds', rounds_temp)

amprabiEF2.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts+1,
    reps=reps_temp*rounds_temp,
    rounds=1,
    sigma_test=pi_lenEF, # gaussian sigma for pulse length - default from cfg [us]
    pulse_ge=False, # set to false when measuring qubit temp
    pulse_type='gauss',
    checkEF=True, 
    qTest=qubit_i,
    qZZ=None,


    # readout_cool=False,
    readout_cool=True,
    n_init_readout=1,
    n_trig=1,
    init_read_wait_us=2,
    singleshot_reps=10000,
    
    # use_gf_readout=True,
    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

# print(amprabiEF2.cfg)
amprabiEF2.go(analyze=False, display=False, progress=True, save=False)

reps 500 rounds 200
Running amp rabi EF on Q1 


  0%|          | 0/2 [00:00<?, ?it/s]

thresholds [0, 327.10455367508064, 0, 0]
angles [0, 0.0, 0, 0]


  0%|          | 0/500000 [00:00<?, ?it/s]

(4, 5, 100000)


In [30]:
post_select = False
# post_select = True
ps_threshold = None
# ps_threshold = 450

In [34]:
# TESTING
# amprabiEF_test = meas.AmplitudeRabiExperiment(config_file=config_path,)
# amprabiEF2 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00004_amp_rabi_EF_qubit1.h5')
# amprabiEF2.data = temp_data

# amprabiEF2.analyze(fit=False)
# amprabiEF2.display(fit=False)

# fitparams = [None]*4
# fitparams[1] = 2/amprabiEF2.data['xpts'][-1]

# amprabiEF2_fit, amprabiEF2_fit_err = meas.fitting.get_best_fit(amprabiEF2.data)

# NATURAL CONSTANTS
h = 6.62607015e-34
fge = 1e6*(np.reshape(amprabiEF2.cfg.device.qubit.f_ge, (4,4))[qubit_i, qubit_i] + amprabiEF2.cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i])
kB = 1.380649e-23
################

fit_axis = 'amps'
# fit_axis = 'avgi' if post_select else 'amps'

amprabiEF.analyze(fit=False, post_select=post_select, ps_threshold=ps_threshold)
amprabiEF_data = amprabiEF.data[fit_axis]
# amprabiEF2_data = (amprabiEF2.data[fit_axis] + temp_data[fit_axis])/2
# p = amprabiEF.data[f'fit_{fit_axis}']
# efamp_normal = p[0]
efamp_normal = (max(amprabiEF.data[fit_axis]) - min(amprabiEF.data[fit_axis]))

amprabiEF2.analyze(fit=False, post_select=post_select, ps_threshold=ps_threshold)
amprabiEF2_data = amprabiEF2.data[fit_axis]
# p = amprabiEF2.data[f'fit_{fit_axis}']
# efamp_nopulse = p[0]
# fit_vals = meas.fitting.sinfunc(amprabiEF.data['xpts'], *p)
# std_dev_no_pulse = np.std(fit_vals[:10])
# efamp_nopulse = (max(fit_vals) - min(fit_vals))-2*std_dev_no_pulse
efamp_nopulse = (max(amprabiEF2.data[fit_axis]) - min(amprabiEF2.data[fit_axis]))

# amprabiEF2.display(fit=True, fit_func='sin')

print('yscale normal:', efamp_normal)
print('yscale no pulse:', efamp_nopulse)

temp = 1e3*-h*fge/(kB*np.log(efamp_nopulse/(efamp_normal+efamp_nopulse)))

print(f'qubit {qubit_i}')
print('Qubit temp [mK]:', temp)
results_Qi.update({'Qubit temp [mK]':temp})
print('Qubit E thermal population (%):', efamp_nopulse/efamp_normal*100)

amprabiEF2.display(fit=False)
results_Qi.update({'Qubit E thermal population (%)':efamp_nopulse/efamp_normal*100})
print(results_Qi)

IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed

In [64]:
if save_data: saved_files.append(amprabiEF2.save_data())

## EF Ramsey Fringe
Update the ef frequency in <code>cfg.qubit.f_ef</code>.

In [67]:
qubitEF_hwhm = 2
# short_T1 = False

In [68]:
ramseyEF_freq = max((min((np.abs(np.ceil(qubitEF_hwhm)), 3)), 2))

t2EFr = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix="ramseyEF"+f"_qubit{qubit_i}",
    config_file=config_path,
)

t2EFr.cfg = AttrDict(deepcopy(yaml_cfg))

t2EFr.cfg.expt = dict(
    start=0, # wait time tau [us]
    step=soc.cycles2us(10), # [us] make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
    # step=0.05, # [us]
    expts=151,
    ramsey_freq=ramseyEF_freq, # [MHz]
    checkEF=True,
    reps=reps,
    rounds=rounds,
    qTest=qubit_i,
    qZZ=None,
)

try:
    t2EFr.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

Running Ramsey EF on Q2 


  0%|          | 0/10 [00:00<?, ?it/s]

In [69]:
# # TESTING
# # t2EFr = meas.RamseyExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00000_ramsey_qubit2.h5')
# t2EFr.data = temp_data

t2EFr.analyze(fit=True)
t2EFr_fit, t2EFr_fit_err, t2EFr_adjust = meas.fitting.get_best_fit(t2EFr.data, get_best_data_params=['f_adjust_ramsey'])
old_qubit_freq = np.reshape(t2EFr.cfg.device.qubit.f_ef, (4,4))[qubit_i, qubit_i]

if not isinstance(t2EFr_fit, (list, np.ndarray)):
    t2EFr.display(fit=False)
    assert False, 'Fit failed, please manually fix!'
t2EFr.display(fit=True)
yaml_cfg.device.qubit.f_ef = np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))
yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] = old_qubit_freq + t2EFr_adjust[0]
alt_qubit_freq = old_qubit_freq + t2EFr_adjust[1]
print(f'*Set qubit {qubit_i} EF freq to {yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i]}*')
results_Qi.update({'Qubit EF freq':yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
print(f'*New f_ef\n', yaml_cfg.device.qubit.f_ef)
yaml_cfg.device.qubit.f_ef = yaml_cfg.device.qubit.f_ef.flatten().tolist()
print(f'*New f_ef flattened\n{yaml_cfg.device.qubit.f_ef}')
results_Qi.update({'T2R EF':t2EFr_fit[3], 'T2R EF err':np.sqrt(t2EFr_fit_err[3][3])})
print(all_results)

best 2 0.03756389571370712
Current pi pulse frequency: 4572.9368263382785
Fit frequency from amps [MHz]: 1.8773201153500298 +/- 0.0018501903904312882
Suggested new pi pulse frequencies from fit amps [MHz]:
 	4573.059506222929
 	4569.059506222929
T2 Ramsey from fit amps [us]: 23.897974500417465 +/- 6.476811221119381
Current pi pulse frequency: 4572.9368263382785
Fit frequency from I [MHz]: 1.8767301525010227 +/- 0.0015684015075514372
Suggested new pi pulse frequency from fit I [MHz]:
 	4573.060096185777
 	4569.060096185777
T2 Ramsey from fit I [us]: 21.04747230209219 +/- 4.26241782956233
Fit frequency from Q [MHz]: 1.878496369433494 +/- 0.001229869725566894
Suggested new pi pulse frequencies from fit Q [MHz]:
 	4573.058329968845
 	4569.058329968845
T2 Ramsey from fit Q [us]: 19.74783530097964 +/- 2.934531109425013
*Set qubit 2 EF freq to 4573.058329968845*
*New f_ef
 [[3884.62691612    0.            0.            0.        ]
 [   0.         3348.08543317    0.            0.        ]
 [4

In [64]:
if save_data: saved_files.append(t2EFr.save_data())

## Amplitude EF Rabi round 2

In [46]:
pi_lenEF = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]
# pi_lenEF = 0.030
print('qubit', qubit_i, 'pi_lenEF', pi_lenEF)

qubit 3 pi_lenEF 0.03


In [47]:
amprabiEF = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix="amp_rabi_EF"+f"_qubit{qubit_i}",
    config_file=config_path,
)

amprabiEF.cfg = AttrDict(deepcopy(yaml_cfg))

span = min((32000, int(1.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i])))
npts = 101

amprabiEF.cfg.expt = dict(
    start=1, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    rounds=rounds,
    sigma_test=pi_lenEF, # gaussian sigma for pulse length - default from cfg [us]
    pulse_type='gauss',
    checkEF=True, 
    qTest=qubit_i,
    qZZ=None,
)

# print(amprabiEF.cfg)
amprabiEF.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi EF on Q3 


  0%|          | 0/10 [00:00<?, ?it/s]

In [48]:
# TESTING
# amprabiEF = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_EF_qubit1.h5')
# amprabiEF.data = temp_data

# amprabiEF.analyze(fit=False)
# amprabiEF.display(fit=False)

amprabiEF.analyze(fit=True)

amprabiEF_fit, amprabiEF_fit_err = meas.fitting.get_best_fit(amprabiEF.data)

p = amprabiEF_fit

if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabiEF.display(fit=False)
    print('WARNING: Fit pi pulse gain is long, please double check!')
    pi_gain = 30000
else: amprabiEF.display(fit=True)

print(f'*Set EF pi pulse sigma for qubit {qubit_i} to {np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]}*')
yaml_cfg.device.qubit.pulses.pi_ef.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set EF pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]})
print(f'*New pi_ef.gain\n', yaml_cfg.device.qubit.pulses.pi_ef.gain)
yaml_cfg.device.qubit.pulses.pi_ef.gain = yaml_cfg.device.qubit.pulses.pi_ef.gain.flatten().astype(int).tolist()
print(f'*New pi_ef.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.gain}')
print(results_Qi)

best 0 0.017386250836177793
Pi gain from amps data [dac units]: 523
	Pi/2 gain from amps data [dac units]: 261
Pi gain from avgi data [dac units]: 523
	Pi/2 gain from avgi data [dac units]: 261
Pi gain from avgq data [dac units]: 541
	Pi/2 gain from avgq data [dac units]: 270
*Set EF pi pulse sigma for qubit 3 to 0.03*
*Set EF pi pulse gain for qubit 3 to 523*
*New pi_ef.gain
 [[ 1280     0     0     0]
 [    0  1912     0     0]
 [10000 10079 10015     0]
 [  734   718     0   523]]
*New pi_ef.gain flattened
[1280, 0, 0, 0, 0, 1912, 0, 0, 10000, 10079, 10015, 0, 734, 718, 0, 523]
{'T1': 52.3580045090926, 'T1 err': 1.6646251872930162, 'T2R': 25.602583027942746, 'T2R err': 1.5551254720790846, 'Qubit freq': 4381.109690194443, 'Pi pulse gain': 935, 'T2E': 27.649964593090505, 'T2E err': 1.1781089735321566, 'Res chi shift (e - g)': -0.009933774834678388, 'Qubit EF freq': 4208.832479183671, 'Pi EF pulse gain': 523, 'Qubit temp [mK]': 72.74668238714514, 'Qubit E thermal population (%)': 5.882

In [ ]:
if save_data: saved_files.append(amprabiEF.save_data())

## EF Ramsey Fringe Round 2
Update the T2 EF but not the frequency

In [21]:
# yaml_cfg.device.qubit.f_ge[qubit_i] = 145.572034953271
# qubitEF_hwhm = 7
# short_T1 = False

In [49]:
t2EFr_2 = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix="ramseyEF"+f"_qubit{qubit_i}",
    config_file=config_path,
)

t2EFr_2.cfg = AttrDict(deepcopy(yaml_cfg))

t2EFr_2.cfg.expt = dict(
    start=0, # wait time tau [us]
    step=soc.cycles2us(80), # [us] make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
    # step=0.05, # [us]
    expts=151,
    ramsey_freq=ramseyEF_freq/10, # [MHz]
    checkEF=True,
    reps=reps,
    rounds=rounds,
    qTest=qubit_i,
    qZZ=None,
)

try:
    t2EFr_2.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

Running Ramsey EF on Q3 


  0%|          | 0/10 [00:00<?, ?it/s]

In [50]:
# # TESTING
# # t2EFr_2 = meas.RamseyExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00000_ramsey_qubit2.h5')
# t2EFr_2.data = temp_data

t2EFr_2.analyze(fit=True)
t2EFr_2_fit, t2EFr_2_fit_err, t2EFr_2_adjust = meas.fitting.get_best_fit(t2EFr_2.data, get_best_data_params=['f_adjust_ramsey'])
old_qubit_freq = np.reshape(t2EFr_2.cfg.device.qubit.f_ef, (4,4))[qubit_i, qubit_i]

if not isinstance(t2EFr_2_fit, (list, np.ndarray)):
    t2EFr_2.display(fit=False)
    assert False, 'Fit failed, please manually fix!'
t2EFr_2.display(fit=True)

if t2EFr_2_adjust[0] > ramseyEF_freq/10:
    print('WARNING: your EF frequency is not converging!')
else:
    # yaml_cfg.device.qubit.f_ef = np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))
    # yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] = old_qubit_freq + t2EFr_2_adjust[0]
    # print(f'*Set qubit {qubit_i} EF freq to {yaml_cfg.device.qubit.f_ef[qubit_i]}*')
    # print(f'*New f_ef\n', yaml_cfg.device.qubit.f_ef)
    # yaml_cfg.device.qubit.f_ef = yaml_cfg.device.qubit.f_ef.flatten().tolist()
    # print(f'*New f_ef flattened\n{yaml_cfg.device.qubit.f_ef}')
    results_Qi.update({'T2R EF':t2EFr_2_fit[3], 'T2R EF err':np.sqrt(t2EFr_2_fit_err[3][3])})
    # results_Qi.update({'Qubit EF freq':yaml_cfg.device.qubit.f_ef[qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
    print(results_Qi)

best 0 0.024697624518023903
Current pi pulse frequency: 4208.832479183671
Fit frequency from amps [MHz]: 0.19447236532119588 +/- 0.0005790511250366627
Suggested new pi pulse frequencies from fit amps [MHz]:
 	4208.83800681835
 	4208.43800681835
T2 Ramsey from fit amps [us]: 16.628768565911635 +/- 1.0029000270571076
Current pi pulse frequency: 4208.832479183671
Fit frequency from I [MHz]: 0.19447232748960125 +/- 0.0005829717124436521
Suggested new pi pulse frequency from fit I [MHz]:
 	4208.838006856182
 	4208.438006856181
T2 Ramsey from fit I [us]: 16.583131135030893 +/- 1.0040165159864192
Fit frequency from Q [MHz]: 0.19333694222682055 +/- 0.0027252320010871765
Suggested new pi pulse frequencies from fit Q [MHz]:
 	4208.839142241444
 	4208.439142241445
T2 Ramsey from fit Q [us]: 12.838583068549777 +/- 2.85017707132049


 s:\Connie\experiments\qram_tprocv1_expts\experiments\single_qubit\t2_ramsey.py: 626

{'T1': 52.3580045090926, 'T1 err': 1.6646251872930162, 'T2R': 25.602583027942746, 'T2R err': 1.5551254720790846, 'Qubit freq': 4381.109690194443, 'Pi pulse gain': 935, 'T2E': 27.649964593090505, 'T2E err': 1.1781089735321566, 'Res chi shift (e - g)': -0.009933774834678388, 'Qubit EF freq': 4208.832479183671, 'Pi EF pulse gain': 523, 'Qubit temp [mK]': 72.74668238714514, 'Qubit E thermal population (%)': 5.88276844959484, 'T2R EF': 16.628768565911635, 'T2R EF err': 1.0029000270571076}


In [ ]:
if save_data: saved_files.append(t2EFr.save_data())

## EF error amplification

In [ ]:
npulsecalibEF = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptEFQ{qubit_i}",
    config_file=config_path,
)

npulsecalibEF.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalibEF.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=10,
    reps=1000,
    loops=10,
    pulse_type='gauss',
    checkEF=True, 
    qTest=qubit_i,
    qZZ=None,
    test_pi_half=False,
    error_amp=True,

    post_process='scale',
    singleshot_reps=15000,
    # singleshot_reps=1000,
)

print(npulsecalibEF.cfg)
try:
    npulsecalibEF.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalibEF.data
npulsecalibEF.analyze(fit=True)
npulsecalibEF.display(fit=True)

# npulsecalib_fit, npulsecalib_fit_err = meas.fitting.get_best_fit(npulsecalibEF.data)
fit_axis = 'avgi'
npulsecalibEF_fit = data[f'fit_{fit_axis}']
npulsecalibEF_fit_err = data[f'fit_err_{fit_axis}']

angle_err = npulsecalibEF_fit[1]
amp_ratio = (180 - angle_err) / 180
new_amp_ef = np.reshape(npulsecalibEF.cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i] / amp_ratio

yaml_cfg.device.qubit.pulses.pi_ef.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i] = int(round(new_amp_ef))
print(f'*Set EF pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]})
print(f'*New pi_ef.gain\n', yaml_cfg.device.qubit.pulses.pi_ef.gain)
yaml_cfg.device.qubit.pulses.pi_ef.gain = yaml_cfg.device.qubit.pulses.pi_ef.gain.flatten().tolist()
print(f'*New pi_ef.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.gain}')
print(results_Qi)

In [ ]:
if save_data: npulsecalibEF.save_data()

## (Divide len) Calibrate EF $\pi/2$ pulse
this is mostly only for RB

### Amplitude rabi for EF $\pi/2$ pulse (divide len)

In [89]:
pi_lenEF = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]
# pi_len = 0.008
print('qubit', qubit_i, 'pi_lenEF', pi_lenEF)

qubit 2 pi_lenEF 0.015


In [90]:
divide_len = True

In [94]:
amprabi_4 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi_4.cfg = AttrDict(deepcopy(yaml_cfg))

n_pulses = 1
span = min((32000, int(1.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i])))
npts = 151
    
amprabi_4.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    sigma_test=pi_lenEF, # gaussian sigma for pulse length - overrides config [us]
    pulse_type='gauss',
    n_pulses=n_pulses,
    test_pi_half=True,
    divide_len=divide_len,
    qTest=qubit_i,
    qZZ=None,
    checkEF=True, 

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,

    # use_robust_pulses=True,
)
amprabi_4.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi EF on Q2 


  0%|          | 0/10 [00:00<?, ?it/s]

(4, 151, 100)


In [95]:
# # TESTING
# # amprabi_4 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi_4.data = temp_data

amprabi_4.analyze(fit=True)

amprabi_4_fit, amprabi_4_fit_err = meas.fitting.get_best_fit(amprabi_4.data)

p = amprabi_4_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi_4.display(fit=False)
    print(f'WARNING: Fit pi/2 pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi_4.display(fit=True)

yaml_cfg.device.qubit.pulses.pi_ef.half_gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.half_gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set pi/2 EF pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i]})
print(f'*New pi_ef.half_gain\n', yaml_cfg.device.qubit.pulses.pi_ef.half_gain)
yaml_cfg.device.qubit.pulses.pi_ef.half_gain = yaml_cfg.device.qubit.pulses.pi_ef.half_gain.flatten().astype(int).tolist()
print(f'*New pi_ef.half_gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain}')
print(results_Qi)

best 0 0.019591024951801192
Pi gain from amps data [dac units]: 6472
	Pi/2 gain from amps data [dac units]: 3236
Pi gain from avgi data [dac units]: 4312
	Pi/2 gain from avgi data [dac units]: 2156
Pi gain from avgq data [dac units]: 4507
	Pi/2 gain from avgq data [dac units]: 2253
*Set pi/2 EF pulse gain for qubit 2 to 6472*
*New pi_ef.half_gain
 [[1261    0    0    0]
 [   0 1941    0    0]
 [   0    0 6472    0]
 [ 340  340    0  320]]
*New pi_ef.half_gain flattened
[1261, 0, 0, 0, 0, 1941, 0, 0, 0, 0, 6472, 0, 340, 340, 0, 320]
{'Res freq': 7702.328233333334, 'Res freq (yaml)': 452.32823333333357, 'Res kappa': 0.2819786993431654, 'Res chi shift (e - g)': -0.943708609271539, 'Res chi shift (f - g)': -1.9172185430463742, 'Res freq ef (yaml)': 450.4510147902872, 'Pi/2 EF pulse gain': 6472}


In [ ]:
if save_data: saved_files.append(amprabi_4.save_data())

### EF $\pi/2$, -$\pi/2$ experiment

In [142]:
piminuspi_EF = meas.PiMinusPiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"PiMinusPi_EFExpt{qubit_i}",
    config_file=config_path,
)

span_f = 2.0
npts_f = 35
# npts_f = 50

# span_f = 0.0
# npts_f = 1

piminuspi_EF.cfg = AttrDict(deepcopy(yaml_cfg))
piminuspi_EF.cfg.expt = dict(
    start_N=0, # number gate repetitions
    # step_N=5,
    # step_N=2,
    # expts_N=15,
    step_N=4,
    expts_N=10,
    start_f=piminuspi_EF.cfg.device.qubit.f_ef[qubit_i*4 + qubit_i]-span_f/2,
    step_f=span_f/npts_f,
    expts_f=npts_f+1,
    reps=int(0.75*reps*rounds),
    loops=1,
    # reps=int(2*reps*rounds),
    # loops=5,
    pulse_type='gauss',

    checkEF=True,
    qTest=qubit_i,
    qZZ=None,

    # use_Y=True,

    test_pi_half=True,
    divide_len=divide_len,

    post_process='threshold',
    singleshot_reps=15000,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

print(piminuspi_EF.cfg)
try:
    piminuspi_EF.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-439.97500000000036, 0.0, 452.3682333333336, -278.89999999999964], 'gain': [0.17, 0.035, 0.25, 0.17], 'kappa_ext': [0.157, 0.065, 0.182, 0.104], 'kerr': [-0.0045, 0, 0.008, -0.003], 't_rise_reset': [0.5, 1, 0.5, 0.5], 'readout_length': [4, 4, 4, 4], 'frequency_ef': [-439.97500000000036, 0.0, 450.44439227373095, -280.30732891832196], 'readout_length_ef': 4, 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4111.097866562327, 4105.838904759637, 4110.958153299016, 4110.546407850519, 3442.8755721075536, 3448.5530242338505, 3447.060317317807, 3446.4929792801067, 4762.761504132457, 4761.823567235975, 4762.909473544484, 4762.369813422167, 4380.698604555338, 4379.604868123916, 4380.717837106693, 4381.248121020163], 'f_ge_robust': [4114.313815659153, 4108.147701523904, 4114.310372455052, 4114.310372455052, 3443.0032303441935, 3448.345849629168, 0, 0, 4763.941, 4763.941, 4763.941, 4763.941, 4384.2503422

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (2)
ge fidelity (%): 79.22666666666663 	 angle (deg): 0.0 	 threshold ge: 186.40066551743595


  0%|          | 0/10 [00:00<?, ?it/s]

In [143]:
# # TESTING
# temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00000_PiMinusPi_EFExpt0.h5')
# piminuspi_EF.data =  temp_data
# piminuspi_EF.cfg = AttrDict(attrs['config'])


data = piminuspi_EF.data

# piminuspi_EF.analyze(fit=False)
# plt.figure()
# plt.plot(data['cycle_sweep'], data['avgi'][:,0], '.-')
# plt.xlabel('N Cycles $(\pi/2, -\pi/2)^N$')
# plt.ylabel('Avg I')
# plt.show()

piminuspi_EF.analyze(fit=True)
piminuspi_EF.display(fit=True)

yaml_cfg.device.qubit.f_ef = np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))
yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] = data["best_freq"]
results_Qi.update({'Qubit EF freq':yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
print(f'*Set qubit {qubit_i} EF freq to {yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i]}*')
print(results_Qi)
print(f'*New f_ef\n', yaml_cfg.device.qubit.f_ef)
yaml_cfg.device.qubit.f_ef = yaml_cfg.device.qubit.f_ef.flatten().tolist()
print(f'*New f_ef flattened\n{yaml_cfg.device.qubit.f_ef}')

Fit best freq 4573.168610947125 which is 0.08048936338036583 away from old freq 4573.088121583744
*Set qubit 2 EF freq to 4573.168610947125*
{'Res freq': 7702.328233333334, 'Res freq (yaml)': 452.32823333333357, 'Res kappa': 0.2819786993431654, 'Res chi shift (e - g)': -0.943708609271539, 'Res chi shift (f - g)': -1.9172185430463742, 'Res freq ef (yaml)': 450.4510147902872, 'Pi/2 EF pulse gain': 6385, 'Qubit EF freq': 4573.168610947125}
*New f_ef
 [[3884.62691612    0.            0.            0.        ]
 [   0.         3348.08543317    0.            0.        ]
 [4571.90564823 4570.99652654 4573.16861095    0.        ]
 [4211.61773253 4205.61697814    0.         4209.92508626]]
*New f_ef flattened
[3884.6269161242526, 0.0, 0.0, 0.0, 0.0, 3348.085433174199, 0.0, 0.0, 4571.9056482340775, 4570.996526541673, 4573.168610947125, 0.0, 4211.617732525247, 4205.61697813965, 0.0, 4209.925086260656]


In [ ]:
if save_data: piminuspi.save_data()

### EF $\pi/2$ error amplification (divide len)

In [115]:
divide_len = True

In [138]:
npulsecalib_efhalf = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptEF_half_Q{qubit_i}",
    config_file=config_path,
)

npulsecalib_efhalf.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib_efhalf.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=10,
    reps=1000,
    # loops=20,
    loops=5,
    pulse_type='gauss',
    test_pi_half=True,
    divide_len=divide_len,
    error_amp=True,

    checkEF=True,
    qTest=qubit_i,
    qZZ=None,

    # readout_ge=False, # whether to readout at the g/e set point (True) or e/f set point (False)
    readout_ge=True, # whether to readout at the g/e set point (True) or e/f set point (False)

    post_process='scale',
    # post_process=None,
    singleshot_reps=15000,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,

    use_robust_pulses=True,
)

print(npulsecalib_efhalf.cfg)
import Pyro4.util
try:
    npulsecalib_efhalf.acquire(progress=True, debug=True)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-439.97500000000036, 0.0, 452.3682333333336, -278.89999999999964], 'gain': [0.17, 0.035, 0.25, 0.17], 'kappa_ext': [0.157, 0.065, 0.182, 0.104], 'kerr': [-0.0045, 0, 0.008, -0.003], 't_rise_reset': [0.5, 1, 0.5, 0.5], 'readout_length': [4, 4, 4, 4], 'frequency_ef': [-439.97500000000036, 0.0, 450.44439227373095, -280.30732891832196], 'readout_length_ef': 4, 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4111.097866562327, 4105.838904759637, 4110.958153299016, 4110.546407850519, 3442.8755721075536, 3448.5530242338505, 3447.060317317807, 3446.4929792801067, 4762.761504132457, 4761.823567235975, 4762.909473544484, 4762.369813422167, 4380.698604555338, 4379.604868123916, 4380.717837106693, 4381.248121020163], 'f_ge_robust': [4114.313815659153, 4108.147701523904, 4114.310372455052, 4114.310372455052, 3443.0032303441935, 3448.345849629168, 0, 0, 4763.941, 4763.941, 4763.941, 4763.941, 4384.2503422

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (2)
g unrotated averages:
Amps 118.20467411805085 +/- 57.38787388991945
e unrotated averages:
Amps 279.91002155643014 +/- 67.09894931806625
ge fidelity (%): 81.78613640909393 	 angle (deg): 0.0 	 threshold ge: 196.18055470399742
thresholds=[0, 0, 196.18055470399742, 0],
angles=[0, 0, 0.0, 0],
ge_avgs=[array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), array([118.20467412,   0.        , 279.91002156,   0.        ]), array([0., 0., 0., 0.])],
counts_calib=[[13720.0, 1280.0], [1456.0, 13544.0]]


  0%|          | 0/5 [00:00<?, ?it/s]

 s:\Connie\experiments\qram_tprocv1_expts\experiments\clifford_averager_program.py: 1822

In [144]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib_efhalf.data
npulsecalib_efhalf.analyze(fit=True, scale=None)
npulsecalib_efhalf.display(fit=True, scale=None)

# npulsecalib_efhalf_fit, npulsecalib_efhalf_fit_err = meas.fitting.get_best_fit(npulsecalib_efhalf.data)
fit_axis = 'avgi'
npulsecalib_efhalf_fit = data[f'fit_{fit_axis}']
npulsecalib_efhalf_fit_err = data[f'fit_err_{fit_axis}']

angle_err = npulsecalib_efhalf_fit[1]
amp_ratio = (90 - angle_err)/90
new_amp_half_ef = np.reshape(npulsecalib_efhalf.cfg.device.qubit.pulses.pi_ef.half_gain, (4,4))[qubit_i, qubit_i] / amp_ratio

yaml_cfg.device.qubit.pulses.pi_ef.half_gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.half_gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i] = int(round(new_amp_half_ef))
print(f'*Set pi/2 EF pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i]})
print(f'*New pi_ef.half_gain\n', yaml_cfg.device.qubit.pulses.pi_ef.half_gain)
yaml_cfg.device.qubit.pulses.pi_ef.half_gain = yaml_cfg.device.qubit.pulses.pi_ef.half_gain.flatten().astype(int).tolist()
print(f'*New pi_ef.half_gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain}')
print(results_Qi)

fitparams [None, 0.017453292519943295]
Attempted to init fitparam 0 to 0.0, which is out of bounds 0.0 to 0.0. Instead init to 0.0
From amps: adjust amplitude to 6378 / 0.9989780638148239 = 6384.524576690065
From avgi: adjust amplitude to 6378 / 0.9989780638148239 = 6384.524576690065
From avgq: adjust amplitude to 6378 / 1.000193925472444 = 6376.76338314826
*Set pi/2 EF pulse gain for qubit 2 to 6385*
*New pi_ef.half_gain
 [[1261    0    0    0]
 [   0 1941    0    0]
 [   0    0 6385    0]
 [ 340  340    0  320]]
*New pi_ef.half_gain flattened
[1261, 0, 0, 0, 0, 1941, 0, 0, 0, 0, 6385, 0, 340, 340, 0, 320]
{'Res freq': 7702.328233333334, 'Res freq (yaml)': 452.32823333333357, 'Res kappa': 0.2819786993431654, 'Res chi shift (e - g)': -0.943708609271539, 'Res chi shift (f - g)': -1.9172185430463742, 'Res freq ef (yaml)': 450.4510147902872, 'Pi/2 EF pulse gain': 6385, 'Qubit EF freq': 4573.168610947125}


In [ ]:
if save_data: npulsecalib_efhalf.save_data()

## (Divide gain) Calibrate EF $\pi/2$ pulse

### Amplitude rabi for EF $\pi/2$ pulse (divide gain) (use this for RB)

In [ ]:
pi_lenEF = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]
# pi_len = 0.008
print('qubit', qubit_i, 'pi_lenEF', pi_lenEF)

In [ ]:
divide_len = False

In [ ]:
amprabi_4 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi_4.cfg = AttrDict(deepcopy(yaml_cfg))

n_pulses = 1
span = min((32000, int(1.5*0.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i])))
npts = 151
    
amprabi_4.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    sigma_test=pi_lenEF, # gaussian sigma for pulse length - overrides config [us]
    pulse_type='gauss',
    checkEF=True, 
    n_pulses=n_pulses,
    test_pi_half=True,
    divide_len=divide_len,
    qTest=qubit_i,
    qZZ=None,
)
amprabi_4.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
# # TESTING
# # amprabi_4 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi_4.data = temp_data

amprabi_4.analyze(fit=True)

amprabi_4_fit, amprabi_4_fit_err = meas.fitting.get_best_fit(amprabi_4.data)

p = amprabi_4_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi_4.display(fit=False)
    print(f'WARNING: Fit pi/2 pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi_4.display(fit=True)

yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set EF pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i]})
print(f'*New pi_ef.half_gain_pi_sigma\n', yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma)
yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma = yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma.flatten().astype(int).tolist()
print(f'*New pi_ef.half_gain_pi_sigma flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma}')
print(results_Qi)

In [ ]:
if save_data: saved_files.append(amprabi_4.save_data())

### EF $\pi/2$ error amplification (divide gain)

this is mostly only necessary for RB

In [ ]:
divide_len = False

In [ ]:
npulsecalib_efhalf = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptEF_half_Q{qubit_i}",
    config_file=config_path,
)

npulsecalib_efhalf.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib_efhalf.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=10,
    reps=1000,
    loops=20,
    pulse_type='gauss',
    test_pi_half=True,
    divide_len=divide_len,
    error_amp=True,

    checkEF=True, 
    qTest=qubit_i,
    qZZ=None,

    readout_ge=False, # whether to readout at the g/e set point (True) or e/f set point (False)

    post_process='scale',
    singleshot_reps=15000,
)

print(npulsecalib_efhalf.cfg)
import Pyro4.util
try:
    npulsecalib_efhalf.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib_efhalf.data
npulsecalib_efhalf.analyze(fit=True, scale=None)
npulsecalib_efhalf.display(fit=True, scale=None)

# npulsecalib_efhalf_fit, npulsecalib_efhalf_fit_err = meas.fitting.get_best_fit(npulsecalib_efhalf.data)
fit_axis = 'avgi'
npulsecalib_efhalf_fit = data[f'fit_{fit_axis}']
npulsecalib_efhalf_fit_err = data[f'fit_err_{fit_axis}']

angle_err = npulsecalib_efhalf_fit[1]
amp_ratio = (90 - angle_err)/90
new_amp_half_ef = np.reshape(npulsecalib_efhalf.cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma, (4,4))[qubit_i, qubit_i] / amp_ratio

yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i] = int(round(new_amp_half_ef))
print(f'*Set pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i]})
print(f'*New pi_ef.half_gain_pi_sigma\n', yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma)
yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma = yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma.flatten().astype(int).tolist()
print(f'*New pi_ef.half_gain_pi_sigma flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma}')
print(results_Qi)

In [ ]:
if save_data: npulsecalib_efhalf.save_data()

## Resonator chi shift (e/f)

In [53]:
rspecEF_chi = meas.ResonatorSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"resonator_spectroscopy_chi_qubit{qubit_i}",
    config_file=config_path,
)
rspecEF_chi.cfg = AttrDict(deepcopy(yaml_cfg))

span = 1.5 # MHz
npts = 151
rspecEF_chi.cfg.expt = dict(
    start=rspecEF_chi.cfg.device.readout.frequency[qubit_i]-abs(results_Qi['Res chi shift (e - g)'])-span, # [MHz]
    # start=rspecEF_chi.cfg.device.readout.frequency[qubit_i]-span/2, # [MHz]
    step=span/npts,
    expts=npts,
    reps=reps*rounds,
    pulse_e=True, # add ge pi pulse prior to measurement
    pulse_f=True, # add ef pi pulse prior to measurement
    qTest=qubit_i,
    qZZ=None,
)
# rspecEF_chi.cfg.device.readout.relax_delay = 100 # Wait time between experiments [us]
rspecEF_chi.go(analyze=False, display=False, progress=True, save=False)

  0%|          | 0/151 [00:00<?, ?it/s]

In [54]:
# # TESTING
# # rspecEF_chi = meas.ResonatorSpectroscopyExperiment(config_file=config_path,) 
# temp_data, attrs = prev_data(expt_path, '00001_resonator_spectroscopy_qubit2.h5')
# rspecEF_chi.data = temp_data

rspecEF_chi.data = rspecEF_chi.analyze(fit=True)
rspecEF_chi_fit = rspecEF_chi.data['fit']
rspecEF_chi.display(fit=True)
# res_freq_f = rspecEF_chi_fit[0] - yaml_cfg.hw.soc.dacs.readout.mixer_freq[qubit_i]
res_freq_f = rspecEF_chi.data['xpts'][np.argmin(rspecEF_chi.data['amps'])]
print('ef chi shift', yaml_cfg.device.readout.lo_sideband[qubit_i]*(res_freq_f - yaml_cfg.device.readout.frequency[qubit_i]))
results_Qi.update({'Res chi shift (f - g)': yaml_cfg.device.readout.lo_sideband[qubit_i]*(res_freq_f - yaml_cfg.device.readout.frequency[qubit_i])})
results_Qi.update({'Res freq ef (yaml)':res_freq_f})

print('Res freq ef:', res_freq_f)

# yaml_cfg.device.readout.frequency_ef = [0]*4
yaml_cfg.device.readout.frequency_ef[qubit_i] = res_freq_f
# res_freq_f = 6953.5
print(all_results)

electrical delay phase correction (no mux support) (deg) 8.390360052101322
param guess [6969.593333333333, 34847.96666666667, 34847.96666666667, 0, -0.05356822564384194, 0.9169805951031621]
param bounds ([6968.858233995586, 0, 0, -6.283185307179586, -0.08035233846576291, -inf], [6970.328432671082, 1000000000.0, 100000000.0, 6.283185307179586, 0.08035233846576291, 1.0184922474746172])
popt [ 6.96961651e+03  2.58881387e+04  1.21941840e+05  1.88788262e-01
 -4.10683523e-02 -3.49123006e-02]

Freq with minimum transmission: -280.30732891832196
Freq with maximum transmission: -281.08216335540806
From fit:
	f0: 6969.616513747147
	Qi: 25888.138733762476 	 kappa_i/2pi: 0.2692204559556689
	Qe: 121941.84006701718 	 kappa_e/2pi: 0.057155251306005905
	Q0: 21354.581112126678
	kappa [MHz]: 0.3263757072616748
	phi [radians]: 0.18878826179097136
baseline 178.75855001117742
ef chi shift -1.3907284768212094
Res freq ef: -280.2973951434875
[{}, {}, {'Res freq': 7702.328233333334, 'Res freq (yaml)': 452.328

In [ ]:
if save_data: saved_files.append(rspecEF_chi.save_data())

## $T_1$ EF

In [ ]:
t1EF = meas.T1Experiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"t1EF_qubit{qubit_i}",
    config_file=config_path,
)
t1EF.cfg = AttrDict(deepcopy(yaml_cfg))

span = 100
npts = 151

t1EF.cfg.expt = dict(
    start=0, # wait time [us]
    step=span/npts,
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    qTest=qubit_i,
    checkEF=True,
)
t1EF.cfg.device.readout.relax_delay = span # Wait time between experiments [us]
t1EF.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
# # TESTING
# # t1EF = meas.T1Experiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_t1EF_qubit1.h5')
# t1EF.data = temp_data

t1EF.analyze(fit_log=False)

t1EF_fit, t1EF_fit_err = meas.fitting.get_best_fit(t1EF.data) #, fitter.expfunc)

t1EF.display(fit=True, fit_log=False)
t1EF_qubit = t1EF_fit[3]
yaml_cfg.device.readout.relax_delay[qubit_i] = np.round(4*t1EF_qubit)
print(f'*Set readout {qubit_i} relax delay to {yaml_cfg.device.readout.relax_delay[qubit_i]}*')
results_Qi.update({'T1_EF':t1EF_qubit, 'T1_EF err':np.sqrt(t1EF_fit_err[3][3])})
print(f'qubit {qubit_i}', results_Qi)

In [ ]:
if save_data: saved_files.append(t1.save_data())

# Single Shot

In [167]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

## Single shot optimization

In [ ]:
results_Qi['Res kappa'] = 0.2
# print(results_Qi['Res kappa'])

In [76]:
qubit_i = 3

In [ ]:
shotopt = meas.SingleShotOptExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"single_shot_opt_qubit{qubit_i}",
    config_file=config_path,
)
shotopt.cfg = AttrDict(deepcopy(yaml_cfg))

# span_f = 2*abs(results_Qi['Res chi shift (f - g)']) # MHz
# span_f = 2*abs(results_Qi['Res chi shift (e - g)']) # MHz

span_f = 1.0*abs(results_Qi['Res kappa']) # MHz
npts_f = 15
# span_f = 0
# npts_f = 1

span_gain = 0.4
npts_gain = 8
# span_gain = 0.0
# npts_gain = 8

# span_len = 5 # us
# npts_len = 10
span_len = 0 # us
npts_len = 1

shotopt.cfg.expt = dict(
    reps=15000,
    qTest=qubit_i,

    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] - span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] + 0.25*span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i],
    start_f=shotopt.cfg.device.readout.frequency[qubit_i] - span_f/2, # [MHz]
    step_f=span_f/npts_f,
    expts_f=npts_f,

    # start_gain=shotopt.cfg.device.readout.gain[qubit_i]-span_gain/2,
    start_gain=0.2,
    step_gain=span_gain/npts_gain,
    expts_gain=npts_gain + 1,
    # start_gain=0.2,
    # step_gain=0,
    # expts_gain=1,

    start_len=shotopt.cfg.device.readout.readout_length[qubit_i],
    step_len=span_len/npts_len,
    expts_len=npts_len,
    # start_len=7,
    # step_len=span_len/npts_len,
    # expts_len=npts_len + 1,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

print(shotopt.cfg)

shotopt.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
imax = shotopt.analyze()
shotopt.display()
print(shotopt.data['fid'])

freq_opt = shotopt.data['fpts'][imax[0]]
gain_opt = shotopt.data['gainpts'][imax[1]]
len_opt = shotopt.data['lenpts'][imax[2]]

In [ ]:

# Using readlines()
file1 = open('C:\\Users\\slab\\Downloads\\singleshot_opt.txt', 'r')
lines = file1.readlines()
freqs = []
gains = []
lengths = []
fids = []

line = lines[0]
words = line.split()
freq = float(words[1][:-1])
freqs.append(freq)
gain = float(words[3][:-1])
gains.append(gain)
length = float(words[5])
lengths.append(length)

fid_line = lines[1]
fids.append(float(fid_line.split()[3]))
 
# Strips the newline character
for i in range(2, len(lines), 2):
    line = lines[i]
    words = line.split()
    freq = float(words[1][:-1])
    if freq not in freqs: freqs.append(freq)
    gain = float(words[3][:-1])
    if gain not in gains: gains.append(gain)
    length = float(words[5])
    if length not in lengths: lengths.append(length)

    fid_line = lines[i+1]
    fids.append(float(fid_line.split()[3]))
    continue

fids = np.append(fids, [0]*22)
fids = np.reshape(fids, newshape=(len(freqs), len(gains), len(lengths)))

# lenpts = [data['lenpts'][0]]
for g_ind, gain in enumerate(gains):
    for l_ind, l in enumerate(lengths):
        plt.plot(freqs, fids[:,g_ind, l_ind], 'o-', label=f'gain: {gain:.2}, len [us]: {l}')
plt.xlabel('Frequency [MHz]')
plt.ylabel(f'Fidelity [%]')
plt.legend()
plt.show()


In [ ]:
if save_data: saved_files.append(shotopt.save_data())

next qubit

In [ ]:
qubit_i = 2
results_Qi = all_results[qubit_i]
results_Qi['Res kappa'] = 0.2
# print(results_Qi['Res kappa'])

In [ ]:
shotopt = meas.SingleShotOptExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"single_shot_opt_qubit{qubit_i}",
    config_file=config_path,
)
shotopt.cfg = AttrDict(deepcopy(yaml_cfg))

# span_f = 2*abs(results_Qi['Res chi shift (f - g)']) # MHz
# span_f = 2*abs(results_Qi['Res chi shift (e - g)']) # MHz

# span_f = 2*abs(results_Qi['Res kappa']) # MHz
# npts_f = 15

span_f = 3*abs(results_Qi['Res kappa']) # MHz
npts_f = 20

# span_gain = 0.85
# npts_gain = 5
span_gain = 0.6
npts_gain = 6

span_len = 0 # us
npts_len = 1

shotopt.cfg.expt = dict(
    reps=20000,
    qTest=qubit_i,

    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] - 0.75*span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] + 0.25*span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i],
    start_f=shotopt.cfg.device.readout.frequency[qubit_i] - span_f/2, # [MHz]
    step_f=span_f/npts_f,
    expts_f=npts_f,

    # start_gain=int(shotopt.cfg.device.readout.gain[qubit_i]-span_gain/2),
    start_gain=0.1,
    # start_gain=0.10,
    step_gain=span_gain/npts_gain,
    expts_gain=npts_gain + 1,

    start_len=5,
    # step_len=span_len/npts_len,
    # expts_len=npts_len + 1,
    step_len=0,
    expts_len=1,
)

print(shotopt.cfg)

shotopt.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
imax = shotopt.analyze()
shotopt.display()
print(shotopt.data['fid'])

freq_opt = shotopt.data['fpts'][imax[0]]
gain_opt = shotopt.data['gainpts'][imax[1]]
len_opt = shotopt.data['lenpts'][imax[2]]

In [ ]:
if save_data: saved_files.append(shotopt.save_data())

next qubit

In [ ]:
qubit_i = 3
results_Qi = all_results[qubit_i]
results_Qi['Res kappa'] = 0.2
# print(results_Qi['Res kappa'])

In [ ]:
shotopt = meas.SingleShotOptExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"single_shot_opt_qubit{qubit_i}",
    config_file=config_path,
)
shotopt.cfg = AttrDict(deepcopy(yaml_cfg))

# span_f = 2*abs(results_Qi['Res chi shift (f - g)']) # MHz
# span_f = 2*abs(results_Qi['Res chi shift (e - g)']) # MHz

# span_f = 2*abs(results_Qi['Res kappa']) # MHz
# npts_f = 15

span_f = 3*abs(results_Qi['Res kappa']) # MHz
npts_f = 20

# span_gain = 0.85
# npts_gain = 5
span_gain = 0.1
npts_gain = 8

span_len = 0 # us
npts_len = 1

shotopt.cfg.expt = dict(
    reps=15000,
    qTest=qubit_i,

    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] - 0.75*span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] + 0.25*span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i],
    start_f=shotopt.cfg.device.readout.frequency[qubit_i] - span_f/2, # [MHz]
    step_f=span_f/npts_f,
    expts_f=npts_f,

    # start_gain=int(shotopt.cfg.device.readout.gain[qubit_i]-span_gain/2),
    start_gain=0.01,
    # start_gain=0.10,
    step_gain=span_gain/npts_gain,
    expts_gain=npts_gain + 1,

    start_len=8.0,
    # step_len=span_len/npts_len,
    # expts_len=npts_len + 1,
    step_len=0,
    expts_len=1,
)

print(shotopt.cfg)

shotopt.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
imax = shotopt.analyze()
shotopt.display()
print(shotopt.data['fid'])

freq_opt = shotopt.data['fpts'][imax[0]]
gain_opt = shotopt.data['gainpts'][imax[1]]
len_opt = shotopt.data['lenpts'][imax[2]]

In [ ]:
if save_data: saved_files.append(shotopt.save_data())

## Best readout parameters

In [101]:
qubit_i = 1

In [18]:
shot = meas.HistogramExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"single_shot_qubit{qubit_i}",
    config_file=config_path,
)
shot.cfg = AttrDict(deepcopy(yaml_cfg))

shot.cfg.expt = dict(
    reps=20000,
    check_f=False,
    qTest=qubit_i,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

# shot.cfg.device.readout.frequency=freq_opt
# shot.cfg.device.readout.gain[qubit_i]=gain_opt
# print(freq_opt, gain_opt)
# shot.cfg.device.readout.readout_length = len_opt # [us]

# shot.cfg.device.readout.frequency=-807.4996158081035
# shot.cfg.device.readout.gain[qubit_i] = 0.85
# shot.cfg.device.readout.readout_length = 2.0 # [us]
# shot.cfg.device.readout.relax_delay = 70 # Wait time between experiments [us]
print(shot.cfg)

shot.go(analyze=False, display=False, progress=True, save=False)

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-439.97500000000036, 0.040000000000000036, 452.3682333333336, -278.89999999999964], 'gain': [0.17, 0.035, 0.25, 0.17], 'kappa_ext': [0.157, 0.065, 0.182, 0.104], 'kerr': [-0.0045, 0, 0.008, -0.003], 't_rise_reset': [0.5, 1, 0.5, 0.5], 'readout_length': [4, 4, 4, 4], 'frequency_ef': [-440.97500000000036, 0.0, 453.3682333333336, -280.35364238410557], 'readout_length_ef': [10.0, 7.0, 4.0, 8.0], 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4111.097866562327, 4105.838904759637, 4110.958153299016, 4110.546407850519, 3442.8755721075536, 3448.5530242338505, 3447.060317317807, 3446.4929792801067, 4762.761504132457, 4761.823567235975, 4762.909473544484, 4762.369813422167, 4380.698604555338, 4379.604868123916, 4380.717837106693, 4381.248121020163], 'f_ge_robust': [4114.313815659153, 4108.147701523904, 4114.310372455052, 4114.310372455052, 3443.0032303441935, 3448.345849629168, 0, 0, 4763.941, 4763.9

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

In [20]:
# # TESTING
# shot = meas.HistogramExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00001_single_shot_qubit0.h5')
# data = shot.analyze(data=temp_data)
# shot.display(data=data)

shot.analyze(verbose=False)
shot.display(verbose=True, amplitude_mode=shot.cfg.expt.full_mux_expt)
results_Qi.update({'Single shot g-e fidelity (%)': 100*shot.data['fids'][0]})

g unrotated averages:
Amps 191.70523519405978 +/- 58.82735628375032
e unrotated averages:
Amps 258.5506022882627 +/- 61.49961154597744
ge fidelity (%): 43.485000000000035
rotation angle (deg): 0.0
threshold ge: 226.0672892255089


In [ ]:
if save_data: saved_files.append(shot.save_data())

In [ ]:
# TESTING
shot = meas.HistogramExperiment(config_file=config_path)
temp_data, attrs = prev_data('S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00000_single_shot_qubit2.h5')
shot.cfg = AttrDict(attrs['config'])
shot.data = temp_data

In [ ]:
# Checking for drift vs. shot number

data = shot.data

Ies = data['Ie']
Qes = data['Qe']
Igs = data['Ig']
Qgs = data['Qg']

plot_e = False

if plot_e:
    i, q = Ies, Qes
else:
    i, q = Igs, Qgs

amps = np.abs(i+1j*q)
phases = np.angle(i+1j*q)

# plt.plot(amps, '.', label='amps')
# plt.ylabel('Amps [ADC units]')
# plt.xlabel('Shot number')
# plt.legend()
# plt.show()

plt.title(f'Q{shot.cfg.expt.qTest} {"e" if plot_e else "g"}')
plt.plot(phases, '.', label='phases')
plt.axhline(0, color='r')
# plt.axhline(1.5, color='r')
plt.ylabel('Phases [radians]')
plt.xlabel('Shot number')
plt.legend()
plt.show()

plt.plot(i, '.', label='i')
plt.ylabel('I [ADC units]')
plt.xlabel('Shot number')
plt.legend()
plt.show()

plt.plot(q, '.', label='Q')
plt.ylabel('Q [ADC units]')
plt.xlabel('Shot number')
plt.legend()
plt.show()

# Results

In [74]:
print(yaml_cfg)
for Qi, results in enumerate(all_results):
    print(f'Qubit {Qi}')
    for key, value in results.items():
        print(f'\t{key}: {value}')
    print()

for saved_file in saved_files:
    print(saved_file)

print(f'f_ge\n{yaml_cfg.device.qubit.f_ge}')
print(f'pi_ge.gain\n{yaml_cfg.device.qubit.pulses.pi_ge.gain}')
print(f'pi_ge.half_gain\n{yaml_cfg.device.qubit.pulses.pi_ge.half_gain}')
print(f'pi_ge.half_gain_pi_sigma\n{yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma}')
print(f'f_ef\n{yaml_cfg.device.qubit.f_ef}')
print(f'pi_ef.gain\n{yaml_cfg.device.qubit.pulses.pi_ef.gain}')
print(f'pi_ef.half_gain\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain}')
print(f'pi_ef.half_gain_pi_sigma\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma}')
print(f'readout.frequency_ef\n{yaml_cfg.device.readout.frequency_ef}')

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-389.9452717544333, 5795.711666663333, 502.368233333334, -228.85410723], 'gain': [0.3, 0.3, 0.85, 0.85], 'readout_length': [15.0, 10.0, 5.0, 15.0], 'frequency_ef': [-391.80446161500305, 5794.287237348879, 500.35983051204073, -230.41370987900663], 'readout_length_ef': [10.0, 7.0, 4.0, 8.0], 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4111.097866562327, 4105.867592682015, 4110.994688934281, 4110.579369837695, 3443.482349236793, 3448.1402562265434, 3447.706823169816, 3447.5052085761054, 4762.743580751455, 4761.745710010409, 4762.909473544484, 4762.370786469688, 4380.292461267258, 4379.535103914169, 4380.328267473819, 4381.248121020163], 'f_ef': [3884.6269161242526, 0.0, 0.0, 0.0, 0.0, 3348.085433174199, 0.0, 0.0, 4571.9056482340775, 4570.996526541673, 4573.058329968845, 0.0, 4209.8719428597715, 4205.61697813965, 0.0, 4208.990881064611], 'f_EgGf': [2686.53594683, 0, 2017.3, 2414.92850999], '